<a href="https://colab.research.google.com/github/suzukiiichiro/N-Queens/blob/master/02Lua.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. ブルートフォース（力まかせ探索）

In [2]:
#!/usr/bin/env luajit

--[[ 
  Luaで学ぶ「アルゴリズムとデータ構造」
  ステップバイステップでＮ−クイーン問題を最適化
  一般社団法人  共同通信社  情報技術局  鈴木  維一郎(suzuki.iichiro@kyodonews.jp)
  
  １．ブルートフォース（力まかせ探索）
 　全ての可能性のある解の候補を体系的に数え上げ、それぞれの解候補が問題の解と
   なるかをチェックする方法
   (※)各行に１個の王妃を配置する組み合わせを再帰的に列挙組み合わせを生成するだ
   けであって8王妃問題を解いているわけではありません

  実行には luajiit が必要です。
  Macの場合は

  # インストール
  $ brew install luajit ;

  # 確認
  $ which lulajit
  $ /usr/local/bin/luajit

  #実行
  $ luajit Lua01_N-Queen.lua
  または
  $ ./Lua01_N-Queen.lua

  N-Queen の データ配列について
  =============================

  総当たり
  結局全部のケースをやってみる（完全解）

  バックトラック
  とりあえずやってみる。ダメなら戻って別の道を探る


  N-Queen: クイーンの効き筋
  =========================
  クイーンの位置から、縦、横、斜めが効き筋となります。

  　　       column(列)
  row(行)_0___1___2___3___4_
       0|-*-|---|---|-*-|---|
        +-------------------+
       1|---|-*-|---|-*-|---|
        +-------------------+ 
       2|---|---|-*-|-*-|-*-| 
        +-------------------+ 
       3|-*-|-*-|-*-|-Q-|-*-|
        +-------------------+
       4|---|---|-*-|-*-|-*-|
        +-------------------+


  N-Queen: 盤面上で互いのクイーンが効き筋にならないように配置
  ===========================================================

        完成図は以下の通りです。

  　　       column(列)
  row(行)_0___1___2___3___4_
       0|-Q-|---|---|---|---|
        +-------------------+
       1|---|---|---|-Q-|---|
        +-------------------+ 
       2|---|-Q-|---|---|---| 
        +-------------------+ 
       3|---|---|---|---|-Q-|
        +-------------------+
       4|---|---|-Q-|---|---|
        +-------------------+


  効き筋の表現
  ============

  クイーンの位置から下側を走査対象とします。

  　すでに効き筋：FALSE(盤面ではF）
  　配置可能    ：TRUE

  　　       column(列) 
  row(行)_0___1___2___3___4_
       0|---|---|---|---|---| 
        +-------------------+
       1|---|---|---|---|---|
        +-------------------+ 
       2|---|-Q-|---|---|---| 
        +-------------------+ 
       3|-F-|-F-|-F-|---|---|
        +-------------------+
       4|---|-F-|---|-F-|---|
        +-------------------+
                      

  効き筋を三つの配列で表現
  ========================

  ■ 基本：aBoard[row]=col
           aBoard[2  ]=1

  　　       column(列)
  row(行)_0___1___2___3___4_
       0|---|---|---|---|---|
        +-------------------+
       1|---|---|---|---|---|
        +-------------------+ 
       2|---|-Q-|---|---|---| aBoard[2]=1 に配置
        +-------------------+
       3|---|---|---|---|---|
        +-------------------+
       4|---|---|---|---|---|
        +-------------------+


  ■配列1：down[row]

  そのrow(行)にQueenがいる場合はFALSE
                      いない場合はTRUE

  　　       column(列)
  row(行)_0___1___2___3___4_
       0|---|---|---|---|---|
        +-------------------+
       1|---|---|---|---|---|
        +-------------------+ 
       2|---|-Q-|---|---|---| 
        +-------------------+
       3|---|-F-|---|---|---|
        +-------------------+
       4|---|-F-|---|---|---|
        +-------------------+
             down[col(1)]==false (すでに効き筋）


  ■配列２：right[col-row+N-1]
                    right[col-row+N-1]==F
                        Qの場所：col(1)-row(2)+(4-1)=2なので
                        col-row+N-1が２のところがＦとなる 
  　　       column(列)
  row(行)_0___1___2___3___4_
       0|---|---|---|---|---|
        +-------------------+
       1|---|---|---|---|---|
        +-------------------+ 
       2|---|-Q-|---|---|---| 
        +-------------------+ 
       3|---|---|-F-|---|---|
        +-------------------+
       4|---|---|---|-F-|---|
        +-------------------+
                      right[col-row+(N-1)]==false(すでに効き筋）


  ■配列3：left[col+row]
                      left[col+row]==F 
                          Qの場所：col(1)+row(2)=3なので
                          col+rowが3になるところがFとなる。

  　　       column(列) 
  row(行)_0___1___2___3___4_
       0|---|---|---|---|---|
        +-------------------+
       1|---|---|---|---|---|
        +-------------------+ 
       2|---|-Q-|---|---|---| 
        +-------------------+ 
       3|-F-|---|---|---|---|
        +-------------------+
       4|---|---|---|---|---|
        +-------------------+
      left[col+row]


  ステップ１
  ==========
  row=0, col=0 にクイーンを配置してみます。

  aBoard[row]=col
     ↓
  aBoard[0]=0;

  　　       column(列) 
  row(行)_0___1___2___3___4_
   ->  0|-Q-|---|---|---|---| aBoard[row]=col
        +-------------------+ aBoard[0  ]=0  
       1|---|---|---|---|---|
        +-------------------+ 
       2|---|---|---|---|---| 
        +-------------------+ 
       3|---|---|---|---|---|
        +-------------------+
       4|---|---|---|---|---|
        +-------------------+


  考え方：２
  ==========
  効き筋を埋めます

  　　       column(列) 
  row(行)_0___1___2___3___4_
   ->  0|-Q-|---|---|---|---| 
        +-------------------+ 
       1|-F-|-F-|---|---|---|
        +-------------------+ left はありません
       2|-F-|---|-F-|---|---| 
        +-------------------+ 
       3|-F-|---|---|-F-|---|
        +-------------------+
       4|-F-|---|---|---|-F-|
        +-------------------+
        down[col]      right[col-row+(N-1)]


  考え方：３
  ==========
  rowが一つ下に降りて０から１となります。
  次の候補は以下のＡ，Ｂ，Ｃとなります

  　　       column(列) 
  row(行)_0___1___2___3___4_
       0|-Q-|---|---|---|---| 
        +-------------------+ 
   ->  1|-F-|-F-|-A-|-B-|-C-|
        +-------------------+ 
       2|-F-|---|-F-|---|---| 
        +-------------------+ 
       3|-F-|---|---|-F-|---|
        +-------------------+
       4|-F-|---|---|---|-F-|
        +-------------------+

  考え方：４
  ==========
  Ａにおいてみます。
  効き筋は以下の通りです。

  　　       column(列) 
  row(行)_0___1___2___3___4_
       0|-Q-|---|---|---|---| 
        +-------------------+ 
   ->  1|-F-|-F-|-Q-|---|---|
        +-------------------+ 
       2|-F-|-F-|-F-|-F-|---| 
        +-------------------+ 
       3|-F-|---|-F-|-F-|-F-| right[col-row+(N-q)]
        +-------------------+
       4|-F-|---|-F-|---|-F-|
        +-------------------+
  left[col+row]  down[col]


  考え方：５
  ==========
  rowが一つ下に降りて１から２となります。
  次の候補はＡとなります

  　　       column(列) 
  row(行)_0___1___2___3___4_
       0|-Q-|---|---|---|---| 
        +-------------------+ 
       1|-F-|-F-|-Q-|---|---|
        +-------------------+ 
   ->  2|-F-|-F-|-F-|-F-|-A-| 
        +-------------------+ 
       3|-F-|---|-F-|-F-|-F-| 
        +-------------------+
       4|-F-|---|-F-|---|-F-|
        +-------------------+

  考え方：６
  ==========
  効き筋は以下の通りです。
  特に加わるところはありません。

  　　       column(列) 
  row(行)_0___1___2___3___4_
       0|-Q-|---|---|---|---| 
        +-------------------+ 
       1|-F-|-F-|-Q-|---|---|
        +-------------------+ 
   ->  2|-F-|-F-|-F-|-F-|-Q-| 
        +-------------------+ 
       3|-F-|---|-F-|-F-|-F-| 
        +-------------------+
       4|-F-|---|-F-|---|-F-|
        +-------------------+

  考え方：７
  ==========
  rowが一つ下に降りて２から３となります。
  次の候補はＡとなります

  　　       column(列) 
  row(行)_0___1___2___3___4_
       0|-Q-|---|---|---|---| 
        +-------------------+ 
       1|-F-|-F-|-Q-|---|---|
        +-------------------+ 
       2|-F-|-F-|-F-|-F-|-Q-| 
        +-------------------+ 
   ->  3|-F-|-A-|-F-|-F-|-F-| 
        +-------------------+
       4|-F-|---|-F-|---|-F-|
        +-------------------+


  考え方：８
  ==========
  効き筋は以下の通りです。

  　　       column(列) 
  row(行)_0___1___2___3___4_
       0|-Q-|---|---|---|---| 
        +-------------------+ 
       1|-F-|-F-|-Q-|---|---|
        +-------------------+ 
       2|-F-|-F-|-F-|-F-|-Q-| 
        +-------------------+ 
   ->  3|-F-|-Q-|-F-|-F-|-F-| 
        +-------------------+
       4|-F-|-F-|-F-|---|-F-|
        +-------------------+


  考え方：９
  ==========
  今回は、うまくいっていますが、
  次の候補がなければ、キャンセルして、
  前のコマを次の候補にコマを移動し、
  処理を継続します。


  考え方：１０
  =========-=

  rowが一つ下に降りて３から４となります。
  候補はのこり１箇所しかありません。

  　　       column(列) 
  row(行)_0___1___2___3___4_
       0|-Q-|---|---|---|---| 
        +-------------------+ 
       1|-F-|-F-|-Q-|---|---|
        +-------------------+ 
       2|-F-|-F-|-F-|-F-|-Q-| 
        +-------------------+ 
       3|-F-|-Q-|-F-|-F-|-F-| 
        +-------------------+
   ->  4|-F-|-F-|-F-|-A-|-F-|
        +-------------------+



  考え方：１１
  ==========
  最後のクイーンをおきます
  columnの最終列は効き筋を確認する必要はありませんね。

  　　       column(列) 
  row(行)_0___1___2___3___4_
       0|-Q-|---|---|---|---| 
        +-------------------+ 
       1|-F-|-F-|-Q-|---|---|
        +-------------------+ 
       2|-F-|-F-|-F-|-F-|-Q-| 
        +-------------------+ 
       3|-F-|-Q-|-F-|-F-|-F-| 
        +-------------------+
   ->  4|-F-|-F-|-F-|-Q-|-F-|
        +-------------------+

  考え方：１２
  ==========
  rowの脇にcolの位置を示します。

  　　       column(列) 
  row(行)_0___1___2___3___4_
       0|-Q-|---|---|---|---|  [0]
        +-------------------+ 
       1|-F-|-F-|-Q-|---|---|  [2]
        +-------------------+ 
       2|-F-|-F-|-F-|-F-|-Q-|  [4]
        +-------------------+ 
       3|-F-|-Q-|-F-|-F-|-F-|  [1]
        +-------------------+
   ->  4|-F-|-F-|-F-|-Q-|-F-|  [3]
        +-------------------+


  考え方：１３
  ==========

  ボード配列は以下のように表します。
  aBoard[]={0,2,4,1,3]

  出力：
    1: 0 0 0 1
    2: 0 0 0 2
    3: 0 0 0 4
    :
    :



  実行結果
  :
  :
  7 7 7 7 7 7 6 7 : 16777208
  7 7 7 7 7 7 7 0 : 16777209
  7 7 7 7 7 7 7 1 : 16777210
  7 7 7 7 7 7 7 2 : 16777211
  7 7 7 7 7 7 7 3 : 16777212
  7 7 7 7 7 7 7 4 : 16777213
  7 7 7 7 7 7 7 5 : 16777214
  7 7 7 7 7 7 7 6 : 16777215
  7 7 7 7 7 7 7 7 : 16777216
]]--

NQueen={}; NQueen.new=function()
  local this={                    --NQueenクラスのローカルメソッド
    board={};
    size=8;
    count=1;
  };
  -- コメントはハイフンをふたつ繋ぎます
  function NQueen:display()       --出力用メソッド
    for col=0,self.size-1,1 do
      io.write(string.format('%2d', self.board[col]));
    end
    print(" : "..self.count);
    self.count=self.count+1;      --インクリメント
  end
  -- 
  function NQueen:NQueen(row)     --メインロジックメソッド
    if row==self.size then        --全列に配置完了 最後の列で出力
      self:display();
    else
      for col=0,self.size-1,1 do  -- 各列にひとつのクイーンを配置する
        self.board[row]=col;
        self:NQueen(row+1);       -- 次の列に王妃を配置
      end
    end
  end
  --
  return setmetatable( this,{__index=NQueen} );
end
--
NQueen.new():NQueen(0);           -- ０列目に王妃を配置してスタート



UsageError: Cell magic `%%lua` not found.


# 2.配置フラグ（制約テスト高速化）

In [ ]:
#!/usr/bin/env luajit

--[[
  Luaで学ぶアルゴリズムとデータ構造  
  ステップバイステップでＮ−クイーン問題を最適化
  一般社団法人  共同通信社  情報技術局  鈴木  維一郎(suzuki.iichiro@kyodonews.jp)
  
  ２．配置フラグ（制約テスト高速化）
   パターンを生成し終わってからチェックを行うのではなく、途中で制約を満たさな
   い事が明らかな場合は、それ以降のパターン生成を行わない。
  「手を進められるだけ進めて、それ以上は無理（それ以上進めても解はない）という
  事がわかると一手だけ戻ってやり直す」という考え方で全ての手を調べる方法。
  (※)各行列に一個の王妃配置する組み合わせを再帰的に列挙分枝走査を行っても、組
  み合わせを列挙するだけであって、8王妃問題を解いているわけではありません。


  N-Queen の データ配列について
  =============================

  総当たり
  結局全部のケースをやってみる（完全解）

  バックトラック
  とりあえずやってみる。ダメなら戻って別の道を探る


  N-Queen: クイーンの効き筋
  =========================
  クイーンの位置から、縦、横、斜めが効き筋となります。

  　　       column(列)
  row(行)_0___1___2___3___4_
       0|-*-|---|---|-*-|---|
        +-------------------+
       1|---|-*-|---|-*-|---|
        +-------------------+ 
       2|---|---|-*-|-*-|-*-| 
        +-------------------+ 
       3|-*-|-*-|-*-|-Q-|-*-|
        +-------------------+
       4|---|---|-*-|-*-|-*-|
        +-------------------+


  N-Queen: 盤面上で互いのクイーンが効き筋にならないように配置
  ===========================================================

        完成図は以下の通りです。

  　　       column(列)
  row(行)_0___1___2___3___4_
       0|-Q-|---|---|---|---|
        +-------------------+
       1|---|---|---|-Q-|---|
        +-------------------+ 
       2|---|-Q-|---|---|---| 
        +-------------------+ 
       3|---|---|---|---|-Q-|
        +-------------------+
       4|---|---|-Q-|---|---|
        +-------------------+


  効き筋の表現
  ============

  クイーンの位置から下側を走査対象とします。

  　すでに効き筋：FALSE(盤面ではF）
  　配置可能    ：TRUE

  　　       column(列) 
  row(行)_0___1___2___3___4_
       0|---|---|---|---|---| 
        +-------------------+
       1|---|---|---|---|---|
        +-------------------+ 
       2|---|-Q-|---|---|---| 
        +-------------------+ 
       3|-F-|-F-|-F-|---|---|
        +-------------------+
       4|---|-F-|---|-F-|---|
        +-------------------+
                      

  効き筋を三つの配列で表現
  ========================

  ■ 基本：aBoard[row]=col
           aBoard[2  ]=1

  　　       column(列)
  row(行)_0___1___2___3___4_
       0|---|---|---|---|---|
        +-------------------+
       1|---|---|---|---|---|
        +-------------------+ 
       2|---|-Q-|---|---|---| aBoard[2]=1 に配置
        +-------------------+
       3|---|---|---|---|---|
        +-------------------+
       4|---|---|---|---|---|
        +-------------------+


  ■配列1：down[row]

  そのrow(行)にQueenがいる場合はFALSE
                      いない場合はTRUE

  　　       column(列)
  row(行)_0___1___2___3___4_
       0|---|---|---|---|---|
        +-------------------+
       1|---|---|---|---|---|
        +-------------------+ 
       2|---|-Q-|---|---|---| 
        +-------------------+
       3|---|-F-|---|---|---|
        +-------------------+
       4|---|-F-|---|---|---|
        +-------------------+
             down[col(1)]==false (すでに効き筋）


  ■配列２：right[col-row+N-1]
                    right[col-row+N-1]==F
                        Qの場所：col(1)-row(2)+(4-1)=2なので
                        col-row+N-1が２のところがＦとなる 
  　　       column(列)
  row(行)_0___1___2___3___4_
       0|---|---|---|---|---|
        +-------------------+
       1|---|---|---|---|---|
        +-------------------+ 
       2|---|-Q-|---|---|---| 
        +-------------------+ 
       3|---|---|-F-|---|---|
        +-------------------+
       4|---|---|---|-F-|---|
        +-------------------+
                      right[col-row+(N-1)]==false(すでに効き筋）


  ■配列3：left[col+row]
                      left[col+row]==F 
                          Qの場所：col(1)+row(2)=3なので
                          col+rowが3になるところがFとなる。

  　　       column(列) 
  row(行)_0___1___2___3___4_
       0|---|---|---|---|---|
        +-------------------+
       1|---|---|---|---|---|
        +-------------------+ 
       2|---|-Q-|---|---|---| 
        +-------------------+ 
       3|-F-|---|---|---|---|
        +-------------------+
       4|---|---|---|---|---|
        +-------------------+
      left[col+row]


  ステップ１
  ==========
  row=0, col=0 にクイーンを配置してみます。

  aBoard[row]=col
     ↓
  aBoard[0]=0;

  　　       column(列) 
  row(行)_0___1___2___3___4_
   ->  0|-Q-|---|---|---|---| aBoard[row]=col
        +-------------------+ aBoard[0  ]=0  
       1|---|---|---|---|---|
        +-------------------+ 
       2|---|---|---|---|---| 
        +-------------------+ 
       3|---|---|---|---|---|
        +-------------------+
       4|---|---|---|---|---|
        +-------------------+


  考え方：２
  ==========
  効き筋を埋めます

  　　       column(列) 
  row(行)_0___1___2___3___4_
   ->  0|-Q-|---|---|---|---| 
        +-------------------+ 
       1|-F-|-F-|---|---|---|
        +-------------------+ left はありません
       2|-F-|---|-F-|---|---| 
        +-------------------+ 
       3|-F-|---|---|-F-|---|
        +-------------------+
       4|-F-|---|---|---|-F-|
        +-------------------+
        down[col]      right[col-row+(N-1)]


  考え方：３
  ==========
  rowが一つ下に降りて０から１となります。
  次の候補は以下のＡ，Ｂ，Ｃとなります

  　　       column(列) 
  row(行)_0___1___2___3___4_
       0|-Q-|---|---|---|---| 
        +-------------------+ 
   ->  1|-F-|-F-|-A-|-B-|-C-|
        +-------------------+ 
       2|-F-|---|-F-|---|---| 
        +-------------------+ 
       3|-F-|---|---|-F-|---|
        +-------------------+
       4|-F-|---|---|---|-F-|
        +-------------------+

  考え方：４
  ==========
  Ａにおいてみます。
  効き筋は以下の通りです。

  　　       column(列) 
  row(行)_0___1___2___3___4_
       0|-Q-|---|---|---|---| 
        +-------------------+ 
   ->  1|-F-|-F-|-Q-|---|---|
        +-------------------+ 
       2|-F-|-F-|-F-|-F-|---| 
        +-------------------+ 
       3|-F-|---|-F-|-F-|-F-| right[col-row+(N-q)]
        +-------------------+
       4|-F-|---|-F-|---|-F-|
        +-------------------+
  left[col+row]  down[col]


  考え方：５
  ==========
  rowが一つ下に降りて１から２となります。
  次の候補はＡとなります

  　　       column(列) 
  row(行)_0___1___2___3___4_
       0|-Q-|---|---|---|---| 
        +-------------------+ 
       1|-F-|-F-|-Q-|---|---|
        +-------------------+ 
   ->  2|-F-|-F-|-F-|-F-|-A-| 
        +-------------------+ 
       3|-F-|---|-F-|-F-|-F-| 
        +-------------------+
       4|-F-|---|-F-|---|-F-|
        +-------------------+

  考え方：６
  ==========
  効き筋は以下の通りです。
  特に加わるところはありません。

  　　       column(列) 
  row(行)_0___1___2___3___4_
       0|-Q-|---|---|---|---| 
        +-------------------+ 
       1|-F-|-F-|-Q-|---|---|
        +-------------------+ 
   ->  2|-F-|-F-|-F-|-F-|-Q-| 
        +-------------------+ 
       3|-F-|---|-F-|-F-|-F-| 
        +-------------------+
       4|-F-|---|-F-|---|-F-|
        +-------------------+

  考え方：７
  ==========
  rowが一つ下に降りて２から３となります。
  次の候補はＡとなります

  　　       column(列) 
  row(行)_0___1___2___3___4_
       0|-Q-|---|---|---|---| 
        +-------------------+ 
       1|-F-|-F-|-Q-|---|---|
        +-------------------+ 
       2|-F-|-F-|-F-|-F-|-Q-| 
        +-------------------+ 
   ->  3|-F-|-A-|-F-|-F-|-F-| 
        +-------------------+
       4|-F-|---|-F-|---|-F-|
        +-------------------+


  考え方：８
  ==========
  効き筋は以下の通りです。

  　　       column(列) 
  row(行)_0___1___2___3___4_
       0|-Q-|---|---|---|---| 
        +-------------------+ 
       1|-F-|-F-|-Q-|---|---|
        +-------------------+ 
       2|-F-|-F-|-F-|-F-|-Q-| 
        +-------------------+ 
   ->  3|-F-|-Q-|-F-|-F-|-F-| 
        +-------------------+
       4|-F-|-F-|-F-|---|-F-|
        +-------------------+


  考え方：９
  ==========
  今回は、うまくいっていますが、
  次の候補がなければ、キャンセルして、
  前のコマを次の候補にコマを移動し、
  処理を継続します。


  考え方：１０
  =========-=

  rowが一つ下に降りて３から４となります。
  候補はのこり１箇所しかありません。

  　　       column(列) 
  row(行)_0___1___2___3___4_
       0|-Q-|---|---|---|---| 
        +-------------------+ 
       1|-F-|-F-|-Q-|---|---|
        +-------------------+ 
       2|-F-|-F-|-F-|-F-|-Q-| 
        +-------------------+ 
       3|-F-|-Q-|-F-|-F-|-F-| 
        +-------------------+
   ->  4|-F-|-F-|-F-|-A-|-F-|
        +-------------------+



  考え方：１１
  ==========
  最後のクイーンをおきます
  columnの最終列は効き筋を確認する必要はありませんね。

  　　       column(列) 
  row(行)_0___1___2___3___4_
       0|-Q-|---|---|---|---| 
        +-------------------+ 
       1|-F-|-F-|-Q-|---|---|
        +-------------------+ 
       2|-F-|-F-|-F-|-F-|-Q-| 
        +-------------------+ 
       3|-F-|-Q-|-F-|-F-|-F-| 
        +-------------------+
   ->  4|-F-|-F-|-F-|-Q-|-F-|
        +-------------------+

  考え方：１２
  ==========
  rowの脇にcolの位置を示します。

  　　       column(列) 
  row(行)_0___1___2___3___4_
       0|-Q-|---|---|---|---|  [0]
        +-------------------+ 
       1|-F-|-F-|-Q-|---|---|  [2]
        +-------------------+ 
       2|-F-|-F-|-F-|-F-|-Q-|  [4]
        +-------------------+ 
       3|-F-|-Q-|-F-|-F-|-F-|  [1]
        +-------------------+
   ->  4|-F-|-F-|-F-|-Q-|-F-|  [3]
        +-------------------+


  考え方：１３
  ==========

  ボード配列は以下のように表します。
  aBoard[]={0,2,4,1,3]

  出力：
    1: 0 0 0 1
    2: 0 0 0 2
    3: 0 0 0 4
    :
    :






  実行結果
  :
  :
  7 6 5 4 2 0 3 1 : 40310
  7 6 5 4 2 1 0 3 : 40311
  7 6 5 4 2 1 3 0 : 40312
  7 6 5 4 2 3 0 1 : 40313
  7 6 5 4 2 3 1 0 : 40314
  7 6 5 4 3 0 1 2 : 40315
  7 6 5 4 3 0 2 1 : 40316
  7 6 5 4 3 1 0 2 : 40317
  7 6 5 4 3 1 2 0 : 40318
  7 6 5 4 3 2 0 1 : 40319
  7 6 5 4 3 2 1 0 : 40320
]]--

NQueen={}; NQueen.new=function()
  --
  local this={
    board={};
    flag={};
    size=8;
    count=1;
  };
  --
  function NQueen:display()       --出力用メソッド
    for col=0,self.size-1,1 do
      io.write(string.format('%2d', self.board[col]));
    end
    print(" : "..self.count);
    self.count=self.count+1;      --インクリメント
  end
  --
  function NQueen:NQueen(row)     --メインロジックメソッド 
    if row==self.size then        --全列に配置完了 最後の列で出力
      self:display();             --出力
    else
      for col=0,self.size-1,1 do  --各列にひとつのクイーンを配置する
        if self.flag[col] then    --i行には王妃は未配置
        else
          self.board[row]=col;    --王妃をi行に配置
          self.flag[col]=true;    --i行に王妃を配置したらtrueに
          self:NQueen(row+1);     --次の列に王妃を配置
          self.flag[col]=false;   --戻ってきたら王妃を取り除く
        end
      end
    end
  end
  --
  return setmetatable( this,{__index=NQueen} );
end
--
NQueen.new():NQueen(0);           -- ０列目に王妃を配置してスタート


# 3.バックトラック                             

In [ ]:
#!/usr/bin/env luajit

--[[
  Luaで学ぶアルゴリズムとデータ構造  
  ステップバイステップでＮ−クイーン問題を最適化
  一般社団法人  共同通信社  情報技術局  鈴木  維一郎(suzuki.iichiro@kyodonews.jp)
 
  ３．バックトラック
   　各列、対角線上にクイーンがあるかどうかのフラグを用意し、途中で制約を満た
   さない事が明らかな場合は、それ以降のパターン生成を行わない。
   　各列、対角線上にクイーンがあるかどうかのフラグを用意することで高速化を図る。
   　これまでは行方向と列方向に重複しない組み合わせを列挙するものですが、王妃
   は斜め方向のコマをとることができるので、どの斜めライン上にも王妃をひとつだ
   けしか配置できない制限を加える事により、深さ優先探索で全ての葉を訪問せず木
   を降りても解がないと判明した時点で木を引き返すということができます。


  N-Queen の データ配列について
  =============================

  総当たり
  結局全部のケースをやってみる（完全解）

  バックトラック
  とりあえずやってみる。ダメなら戻って別の道を探る


  N-Queen: クイーンの効き筋
  =========================
  クイーンの位置から、縦、横、斜めが効き筋となります。

  　　       column(列)
  row(行)_0___1___2___3___4_
       0|-*-|---|---|-*-|---|
        +-------------------+
       1|---|-*-|---|-*-|---|
        +-------------------+ 
       2|---|---|-*-|-*-|-*-| 
        +-------------------+ 
       3|-*-|-*-|-*-|-Q-|-*-|
        +-------------------+
       4|---|---|-*-|-*-|-*-|
        +-------------------+


  N-Queen: 盤面上で互いのクイーンが効き筋にならないように配置
  ===========================================================

        完成図は以下の通りです。

  　　       column(列)
  row(行)_0___1___2___3___4_
       0|-Q-|---|---|---|---|
        +-------------------+
       1|---|---|---|-Q-|---|
        +-------------------+ 
       2|---|-Q-|---|---|---| 
        +-------------------+ 
       3|---|---|---|---|-Q-|
        +-------------------+
       4|---|---|-Q-|---|---|
        +-------------------+


  効き筋の表現
  ============

  クイーンの位置から下側を走査対象とします。

  　すでに効き筋：FALSE(盤面ではF）
  　配置可能    ：TRUE

  　　       column(列) 
  row(行)_0___1___2___3___4_
       0|---|---|---|---|---| 
        +-------------------+
       1|---|---|---|---|---|
        +-------------------+ 
       2|---|-Q-|---|---|---| 
        +-------------------+ 
       3|-F-|-F-|-F-|---|---|
        +-------------------+
       4|---|-F-|---|-F-|---|
        +-------------------+
                      

  効き筋を三つの配列で表現
  ========================

  ■ 基本：aBoard[row]=col
           aBoard[2  ]=1

  　　       column(列)
  row(行)_0___1___2___3___4_
       0|---|---|---|---|---|
        +-------------------+
       1|---|---|---|---|---|
        +-------------------+ 
       2|---|-Q-|---|---|---| aBoard[2]=1 に配置
        +-------------------+
       3|---|---|---|---|---|
        +-------------------+
       4|---|---|---|---|---|
        +-------------------+


  ■配列1：down[row]

  そのrow(行)にQueenがいる場合はFALSE
                      いない場合はTRUE

  　　       column(列)
  row(行)_0___1___2___3___4_
       0|---|---|---|---|---|
        +-------------------+
       1|---|---|---|---|---|
        +-------------------+ 
       2|---|-Q-|---|---|---| 
        +-------------------+
       3|---|-F-|---|---|---|
        +-------------------+
       4|---|-F-|---|---|---|
        +-------------------+
             down[col(1)]==false (すでに効き筋）


  ■配列２：right[col-row+N-1]
                    right[col-row+N-1]==F
                        Qの場所：col(1)-row(2)+(4-1)=2なので
                        col-row+N-1が２のところがＦとなる 
  　　       column(列)
  row(行)_0___1___2___3___4_
       0|---|---|---|---|---|
        +-------------------+
       1|---|---|---|---|---|
        +-------------------+ 
       2|---|-Q-|---|---|---| 
        +-------------------+ 
       3|---|---|-F-|---|---|
        +-------------------+
       4|---|---|---|-F-|---|
        +-------------------+
                      right[col-row+(N-1)]==false(すでに効き筋）


  ■配列3：left[col+row]
                      left[col+row]==F 
                          Qの場所：col(1)+row(2)=3なので
                          col+rowが3になるところがFとなる。

  　　       column(列) 
  row(行)_0___1___2___3___4_
       0|---|---|---|---|---|
        +-------------------+
       1|---|---|---|---|---|
        +-------------------+ 
       2|---|-Q-|---|---|---| 
        +-------------------+ 
       3|-F-|---|---|---|---|
        +-------------------+
       4|---|---|---|---|---|
        +-------------------+
      left[col+row]


  ステップ１
  ==========
  row=0, col=0 にクイーンを配置してみます。

  aBoard[row]=col
     ↓
  aBoard[0]=0;

  　　       column(列) 
  row(行)_0___1___2___3___4_
   ->  0|-Q-|---|---|---|---| aBoard[row]=col
        +-------------------+ aBoard[0  ]=0  
       1|---|---|---|---|---|
        +-------------------+ 
       2|---|---|---|---|---| 
        +-------------------+ 
       3|---|---|---|---|---|
        +-------------------+
       4|---|---|---|---|---|
        +-------------------+


  考え方：２
  ==========
  効き筋を埋めます

  　　       column(列) 
  row(行)_0___1___2___3___4_
   ->  0|-Q-|---|---|---|---| 
        +-------------------+ 
       1|-F-|-F-|---|---|---|
        +-------------------+ left はありません
       2|-F-|---|-F-|---|---| 
        +-------------------+ 
       3|-F-|---|---|-F-|---|
        +-------------------+
       4|-F-|---|---|---|-F-|
        +-------------------+
        down[col]      right[col-row+(N-1)]


  考え方：３
  ==========
  rowが一つ下に降りて０から１となります。
  次の候補は以下のＡ，Ｂ，Ｃとなります

  　　       column(列) 
  row(行)_0___1___2___3___4_
       0|-Q-|---|---|---|---| 
        +-------------------+ 
   ->  1|-F-|-F-|-A-|-B-|-C-|
        +-------------------+ 
       2|-F-|---|-F-|---|---| 
        +-------------------+ 
       3|-F-|---|---|-F-|---|
        +-------------------+
       4|-F-|---|---|---|-F-|
        +-------------------+

  考え方：４
  ==========
  Ａにおいてみます。
  効き筋は以下の通りです。

  　　       column(列) 
  row(行)_0___1___2___3___4_
       0|-Q-|---|---|---|---| 
        +-------------------+ 
   ->  1|-F-|-F-|-Q-|---|---|
        +-------------------+ 
       2|-F-|-F-|-F-|-F-|---| 
        +-------------------+ 
       3|-F-|---|-F-|-F-|-F-| right[col-row+(N-q)]
        +-------------------+
       4|-F-|---|-F-|---|-F-|
        +-------------------+
  left[col+row]  down[col]


  考え方：５
  ==========
  rowが一つ下に降りて１から２となります。
  次の候補はＡとなります

  　　       column(列) 
  row(行)_0___1___2___3___4_
       0|-Q-|---|---|---|---| 
        +-------------------+ 
       1|-F-|-F-|-Q-|---|---|
        +-------------------+ 
   ->  2|-F-|-F-|-F-|-F-|-A-| 
        +-------------------+ 
       3|-F-|---|-F-|-F-|-F-| 
        +-------------------+
       4|-F-|---|-F-|---|-F-|
        +-------------------+

  考え方：６
  ==========
  効き筋は以下の通りです。
  特に加わるところはありません。

  　　       column(列) 
  row(行)_0___1___2___3___4_
       0|-Q-|---|---|---|---| 
        +-------------------+ 
       1|-F-|-F-|-Q-|---|---|
        +-------------------+ 
   ->  2|-F-|-F-|-F-|-F-|-Q-| 
        +-------------------+ 
       3|-F-|---|-F-|-F-|-F-| 
        +-------------------+
       4|-F-|---|-F-|---|-F-|
        +-------------------+

  考え方：７
  ==========
  rowが一つ下に降りて２から３となります。
  次の候補はＡとなります

  　　       column(列) 
  row(行)_0___1___2___3___4_
       0|-Q-|---|---|---|---| 
        +-------------------+ 
       1|-F-|-F-|-Q-|---|---|
        +-------------------+ 
       2|-F-|-F-|-F-|-F-|-Q-| 
        +-------------------+ 
   ->  3|-F-|-A-|-F-|-F-|-F-| 
        +-------------------+
       4|-F-|---|-F-|---|-F-|
        +-------------------+


  考え方：８
  ==========
  効き筋は以下の通りです。

  　　       column(列) 
  row(行)_0___1___2___3___4_
       0|-Q-|---|---|---|---| 
        +-------------------+ 
       1|-F-|-F-|-Q-|---|---|
        +-------------------+ 
       2|-F-|-F-|-F-|-F-|-Q-| 
        +-------------------+ 
   ->  3|-F-|-Q-|-F-|-F-|-F-| 
        +-------------------+
       4|-F-|-F-|-F-|---|-F-|
        +-------------------+


  考え方：９
  ==========
  今回は、うまくいっていますが、
  次の候補がなければ、キャンセルして、
  前のコマを次の候補にコマを移動し、
  処理を継続します。


  考え方：１０
  =========-=

  rowが一つ下に降りて３から４となります。
  候補はのこり１箇所しかありません。

  　　       column(列) 
  row(行)_0___1___2___3___4_
       0|-Q-|---|---|---|---| 
        +-------------------+ 
       1|-F-|-F-|-Q-|---|---|
        +-------------------+ 
       2|-F-|-F-|-F-|-F-|-Q-| 
        +-------------------+ 
       3|-F-|-Q-|-F-|-F-|-F-| 
        +-------------------+
   ->  4|-F-|-F-|-F-|-A-|-F-|
        +-------------------+



  考え方：１１
  ==========
  最後のクイーンをおきます
  columnの最終列は効き筋を確認する必要はありませんね。

  　　       column(列) 
  row(行)_0___1___2___3___4_
       0|-Q-|---|---|---|---| 
        +-------------------+ 
       1|-F-|-F-|-Q-|---|---|
        +-------------------+ 
       2|-F-|-F-|-F-|-F-|-Q-| 
        +-------------------+ 
       3|-F-|-Q-|-F-|-F-|-F-| 
        +-------------------+
   ->  4|-F-|-F-|-F-|-Q-|-F-|
        +-------------------+

  考え方：１２
  ==========
  rowの脇にcolの位置を示します。

  　　       column(列) 
  row(行)_0___1___2___3___4_
       0|-Q-|---|---|---|---|  [0]
        +-------------------+ 
       1|-F-|-F-|-Q-|---|---|  [2]
        +-------------------+ 
       2|-F-|-F-|-F-|-F-|-Q-|  [4]
        +-------------------+ 
       3|-F-|-Q-|-F-|-F-|-F-|  [1]
        +-------------------+
   ->  4|-F-|-F-|-F-|-Q-|-F-|  [3]
        +-------------------+


  考え方：１３
  ==========

  ボード配列は以下のように表します。
  aBoard[]={0,2,4,1,3]

  出力：
    1: 0 0 0 1
    2: 0 0 0 2
    3: 0 0 0 4
    :
    :







 実行結果
 N:            Total       Unique    hh:mm:ss
 2:                0            0    00:00:00
 3:                0            0    00:00:00
 4:                2            0    00:00:00
 5:               10            0    00:00:00
 6:                4            0    00:00:00
 7:               40            0    00:00:00
 8:               92            0    00:00:00
 9:              352            0    00:00:00
10:              724            0    00:00:00
11:             2680            0    00:00:00
12:            14200            0    00:00:00
13:            73712            0    00:00:01
14:           365596            0    00:00:04
15:          2279184            0    00:00:26
16:         14772512            0    00:02:55
17:         95815104            0    00:20:48
]]--

NQueen={}; NQueen.new=function()
  -- 
  local this={                        --クラス変数
    SIZE=0;
    TOTAL=0;
    UNIQUE=0;
    colChk={};
    diagChk={};
    antiChk={};
    board={};
  };
  --
  function NQueen:NQueens(row)        --メインロジックメソッド
    if row==self.SIZE then
      self.TOTAL=self.TOTAL+1 ;
    else
      for col=0,self.SIZE-1,1 do
        self.board[row]=col; 
        if	self.colChk[col]==nil 
          and self.antiChk[row+col]==nil 
          and self.diagChk[row-col+(self.SIZE-1)]==nil then
          self.colChk[col],
          self.diagChk[row-self.board[row]+self.SIZE-1],
          self.antiChk[row+self.board[row]]=true,true,true;
          self:NQueens(row+1);
          self.colChk[col],
          self.diagChk[row-self.board[row]+self.SIZE-1],
          self.antiChk[row+self.board[row]]=nil,nil,nil;
        end
      end
    end
  end
  --
  function NQueen:secstotime(secs)    --計測時間処理メソッド
    sec=math.floor(secs);
	  if(sec>59) then
		  local hour = math.floor(sec*0.000277777778)
		  local minute = math.floor(sec*0.0166666667) - hour*60
		  sec = sec - hour*3600 - minute*60
		  if(sec<10)then sec = "0"..sec end
		  if(hour<10)then hour = "0"..hour end
		  if(minute<10)then minute = "0"..minute end
		  return hour..":"..minute..":"..sec
	  end
	  if(sec<10)then sec = "0"..sec end
	  return "00:00:"..sec
  end 
  --
  function NQueen:NQueen()
    local max=27;
    print(" N:            Total       Unique    hh:mm:ss");
    for si=2,max,1 do
      self.SIZE=si;                 --初期化
      self.TOTAL=0;
      self.UNIQUE=0;
      s=os.time();                  --計測開始
      self:NQueens(0);
      print(string.format("%2d:%17d%13d%12s",si,self.TOTAL,0,self:secstotime(os.difftime(os.time(),s))));            --計測終了とともに出力 
    end
  end
  --
  return setmetatable( this,{__index=NQueen} );
end
--
NQueen.new():NQueen();              --実行



# 4.対称解除法(回転と斜軸）

In [ ]:
#!/usr/bin/env luajit

--[[
  Luaで学ぶアルゴリズムとデータ構造  
  ステップバイステップでＮ−クイーン問題を最適化
  一般社団法人  共同通信社  情報技術局  鈴木  維一郎(suzuki.iichiro@kyodonews.jp)
  
  ４．対称解除法
      一つの解には、盤面を９０度、１８０度、２７０度回転、及びそれらの鏡像の合計
      ８個の対称解が存在する。対照的な解を除去し、ユニーク解から解を求める手法。
  
  ■ユニーク解の判定方法
    全探索によって得られたある１つの解が、回転・反転などによる本質的に変わること
  のない変換によって他の解と同型となるものが存在する場合、それを別の解とはしない
  とする解の数え方で得られる解を「ユニーク解」といいます。つまり、ユニーク解とは、
  全解の中から回転・反転などによる変換によって同型になるもの同士をグループ化する
  ことを意味しています。
  
    従って、ユニーク解はその「個数のみ」に着目され、この解はユニーク解であり、こ
  の解はユニーク解ではないという定まった判定方法はありません。ユニーク解であるか
  どうかの判断はユニーク解の個数を数える目的の為だけに各個人が自由に定義すること
  になります。もちろん、どのような定義をしたとしてもユニーク解の個数それ自体は変
  わりません。
  
    さて、Ｎクイーン問題は正方形のボードで形成されるので回転・反転による変換パター
  ンはぜんぶで８通りあります。だからといって「全解数＝ユニーク解数×８」と単純には
  いきません。ひとつのグループの要素数が必ず８個あるとは限らないのです。Ｎ＝５の
  下の例では要素数が２個のものと８個のものがあります。
 
 
  Ｎ＝５の全解は１０、ユニーク解は２なのです。
  
  グループ１: ユニーク解１つ目
  - - - Q -   - Q - - -
  Q - - - -   - - - - Q
  - - Q - -   - - Q - -
  - - - - Q   Q - - - -
  - Q - - -   - - - Q -
  
  グループ２: ユニーク解２つ目
  - - - - Q   Q - - - -   - - Q - -   - - Q - -   - - - Q -   - Q - - -   Q - - - -   - - - - Q
  - - Q - -   - - Q - -   Q - - - -   - - - - Q   - Q - - -   - - - Q -   - - - Q -   - Q - - -
  Q - - - -   - - - - Q   - - - Q -   - Q - - -   - - - - Q   Q - - - -   - Q - - -   - - - Q -
  - - - Q -   - Q - - -   - Q - - -   - - - Q -   - - Q - -   - - Q - -   - - - - Q   Q - - - -
  - Q - - -   - - - Q -   - - - - Q   Q - - - -   Q - - - -   - - - - Q   - - Q - -   - - Q - -
 
  
    それでは、ユニーク解を判定するための定義付けを行いますが、次のように定義する
  ことにします。各行のクイーンが右から何番目にあるかを調べて、最上段の行から下
  の行へ順番に列挙します。そしてそれをＮ桁の数値として見た場合に最小値になるもの
  をユニーク解として数えることにします。尚、このＮ桁の数を以後は「ユニーク判定値」
  と呼ぶことにします。
  
  - - - - Q   0
  - - Q - -   2
  Q - - - -   4   --->  0 2 4 1 3  (ユニーク判定値)
  - - - Q -   1
  - Q - - -   3
  
  
    探索によって得られたある１つの解(オリジナル)がユニーク解であるかどうかを判定
  するには「８通りの変換を試み、その中でオリジナルのユニーク判定値が最小であるか
  を調べる」ことになります。しかし結論から先にいえば、ユニーク解とは成り得ないこ
  とが明確なパターンを探索中に切り捨てるある枝刈りを組み込むことにより、３通りの
  変換を試みるだけでユニーク解の判定が可能になります。
   
  
  ■ユニーク解の個数を求める
    先ず最上段の行のクイーンの位置に着目します。その位置が左半分の領域にあればユ
  ニーク解には成り得ません。何故なら左右反転によって得られるパターンのユニーク判
  定値の方が確実に小さくなるからです。また、Ｎが奇数の場合に中央にあった場合はど
  うでしょう。これもユニーク解には成り得ません。何故なら仮に中央にあった場合、そ
  れがユニーク解であるためには少なくとも他の外側の３辺におけるクイーンの位置も中
  央になければならず、それは互いの効き筋にあたるので有り得ません。
 
 
  ***********************************************************************
  最上段の行のクイーンの位置は中央を除く右側の領域に限定されます。(ただし、N ≧ 2)
  ***********************************************************************
  
    次にその中でも一番右端(右上の角)にクイーンがある場合を考えてみます。他の３つ
  の角にクイーンを置くことはできないので(効き筋だから）、ユニーク解であるかどうか
  を判定するには、右上角から左下角を通る斜軸で反転させたパターンとの比較だけになり
  ます。突き詰めれば、
  
  [上から２行目のクイーンの位置が右から何番目にあるか]
  [右から２列目のクイーンの位置が上から何番目にあるか]
  
 
  を比較するだけで判定することができます。この２つの値が同じになることはないからです。
  
        3 0
        ↓↓
  - - - - Q ←0
  - Q - - - ←3
  - - - - -         上から２行目のクイーンの位置が右から４番目にある。
  - - - Q -         右から２列目のクイーンの位置が上から４番目にある。
  - - - - -         しかし、互いの効き筋にあたるのでこれは有り得ない。
  
    結局、再帰探索中において下図の X への配置を禁止する枝刈りを入れておけば、得
  られる解は総てユニーク解であることが保証されます。
  
  - - - - X Q
  - Q - - X -
  - - - - X -
  - - - - X -
  - - - - - -
  - - - - - -
  
    次に右端以外にクイーンがある場合を考えてみます。オリジナルがユニーク解である
  ためには先ず下図の X への配置は禁止されます。よって、その枝刈りを先ず入れておき
  ます。
  
  X X - - - Q X X
  X - - - - - - X
  - - - - - - - -
  - - - - - - - -
  - - - - - - - -
  - - - - - - - -
  X - - - - - - X
  X X - - - - X X
  
    次にクイーンの利き筋を辿っていくと、結局、オリジナルがユニーク解ではない可能
  性があるのは、下図の A,B,C の位置のどこかにクイーンがある場合に限られます。従っ
  て、90度回転、180度回転、270度回転の３通りの変換パターンだけを調べれはよいこと
  になります。
  
  X X x x x Q X X
  X - - - x x x X
  C - - x - x - x
  - - x - - x - -
  - x - - - x - -
  x - - - - x - A
  X - - - - x - X
  X X B - - x X X
 
 
  ■ユニーク解から全解への展開
    これまでの考察はユニーク解の個数を求めるためのものでした。全解数を求めるには
  ユニーク解を求めるための枝刈りを取り除いて全探索する必要があります。したがって
  探索時間を犠牲にしてしまうことになります。そこで「ユニーク解の個数から全解数を
  導いてしまおう」という試みが考えられます。これは、左右反転によるパターンの探索
  を省略して最後に結果を２倍するというアイデアの拡張版といえるものです。そしてそ
  れを実現させるには「あるユニーク解が属するグループの要素数はいくつあるのか」と
  いう考察が必要になってきます。
  
    最初に、クイーンが右上角にあるユニーク解を考えます。斜軸で反転したパターンが
  オリジナルと同型になることは有り得ないことと(×２)、右上角のクイーンを他の３つの
  角に写像させることができるので(×４)、このユニーク解が属するグループの要素数は必
  ず８個(＝２×４)になります。
  
    次に、クイーンが右上角以外にある場合は少し複雑になりますが、考察を簡潔にする
  ために次の事柄を確認します。
 
  TOTAL = (COUNT8 * 8) + (COUNT4 * 4) + (COUNT2 * 2);
    (1) 90度回転させてオリジナルと同型になる場合、さらに90度回転(オリジナルか
     ら180度回転)させても、さらに90度回転(オリジナルから270度回転)させてもオリ
     ジナルと同型になる。  
 
     COUNT2 * 2
  
    (2) 90度回転させてオリジナルと異なる場合は、270度回転させても必ずオリジナ
     ルとは異なる。ただし、180度回転させた場合はオリジナルと同型になることも有
     り得る。 
 
     COUNT4 * 4
  
    (3) (1) に該当するユニーク解が属するグループの要素数は、左右反転させたパターンを
        加えて２個しかありません。(2)に該当するユニーク解が属するグループの要素数は、
        180度回転させて同型になる場合は４個(左右反転×縦横回転)、そして180度回転させても
        オリジナルと異なる場合は８個になります。(左右反転×縦横回転×上下反転)
  
     COUNT8 * 8 
 
    以上のことから、ひとつひとつのユニーク解が上のどの種類に該当するのかを調べる
  ことにより全解数を計算で導き出すことができます。探索時間を短縮させてくれる枝刈
  りを外す必要がなくなったというわけです。 
  
    UNIQUE  COUNT2      +  COUNT4      +  COUNT8
    TOTAL  (COUNT2 * 2) + (COUNT4 * 4) + (COUNT8 * 8)
 
  　これらを実現すると、前回のNQueen3()よりも実行速度が遅くなります。
  　なぜなら、対称・反転・斜軸を反転するための処理が加わっているからです。
  ですが、今回の処理を行うことによって、さらにNQueen5()では、処理スピードが飛
  躍的に高速化されます。そのためにも今回のアルゴリズム実装は必要なのです。
 
 実行結果 luajit
 N:            Total       Unique    hh:mm:ss
 2:                0            0    00:00:00
 3:                0            0    00:00:00
 4:                2            1    00:00:00
 5:               10            2    00:00:00
 6:                4            1    00:00:00
 7:               40            6    00:00:00
 8:               92           12    00:00:00
 9:              352           46    00:00:00
10:              724           92    00:00:00
11:             2680          341    00:00:00
12:            14200         1787    00:00:01
13:            73712         9233    00:00:00
14:           365596        45752    00:00:05
15:          2279184       285053    00:00:31
16:         14772512      1846955    00:03:25
17:         95815104     11977939    00:24:31

]]--

NQueen={}; NQueen.new=function()
  --
  local this={
    SIZE=0;
    nTOTAL=0;
    nUNIQUE=0;
    nEquiv=0;
    colChk={};
    diagChk={};
    antiChk={};
    board={};
  };
  --
  function NQueen:secstotime(secs)
    sec=math.floor(secs);
	  if(sec>59) then
		  local hour=math.floor(sec*0.000277777778);
		  local minute=math.floor(sec*0.0166666667)-hour*60;
		  sec=sec-hour*3600-minute*60
		  if(sec<10)then sec="0"..sec; end
		  if(hour<10)then hour="0"..hour; end
		  if(minute<10)then minute="0"..minute; end
		  return hour..":"..minute..":"..sec;
	  end
	  if(sec<10)then sec="0"..sec end
	  return "00:00:"..sec;
  end 
  --
  function NQueen:intncmp(lt,rt,n)
    local k,rtn=0,0; 
    for k=0,n-1,1 do
      rtn=lt[k]-rt[k];
      if (rtn~=0)then break; end
    end
    return rtn;
  end
  --
  function NQueen:rotate(trial,scratch,n,neg)
    local k=0;
	  local incr=0;
    if neg then k=0; else k=n-1; end
    if neg then incr=1; else incr=-1; end 
	  local j=0;
	  while j<n do
      scratch[j]=trial[k];
		  k=k+incr;
      j=j+1;
    end
    if neg then k=n-1; else k=0; end 
	  local j=0;
	  while j<n do
      trial[scratch[j]]=k;
      j=j+1;
	    k=k-incr;
    end
  end
  --
  function NQueen:vMirror(check,n)
    for j=0,n-1,1 do
      check[j]=(n-1)-check[j];
    end
  end
  --
  function NQueen:symmetryOps()
	  local trial={};   --//回転・反転の対称チェック
	  local scratch={};
    self.nEquiv=0; 	  
    for k=0,self.size-1,1 do trial[k]=self.board[k]; end
    --//時計回りに90度回転
    self:rotate(trial,scratch,self.size,nil);
    local k=self:intncmp(self.board,trial,self.size);
    if(k>0)then return 0; end
    if(k==0)then self.nEquiv=1; else
      --//時計回りに180度回転
      self:rotate(trial,scratch,self.size,nil);
      k=self:intncmp(self.board,trial,self.size);
      if(k>0) then return 0; end
      if(k==0)then self.nEquiv=2; else
        --//時計回りに270度回転
        self:rotate(trial,scratch,self.size,nil);
        k=self:intncmp(self.board,trial,self.size);
        if(k>0) then return 0; end
        self.nEquiv=4;
      end
    end  
    for k=0,self.size-1,1 do trial[k]=self.board[k]; end
    --//垂直反転 
    self:vMirror(trial,self.size);	
    k=self:intncmp(self.board,trial,self.size);
    if(k>0) then return 0; end
    --// 4回転とは異なる場合
    if (self.nEquiv>1) then
			-- 90度回転 対角鏡と同等
      self:rotate(trial,scratch,self.size,true);
      k=self:intncmp(self.board,trial,self.size);
      if(k>0) then return 0; end
      --// 2回転とは異なる場合
      if(self.nEquiv>2)then
			  -- 180度回転 水平鏡像と同等
        self:rotate(trial,scratch,self.size,true);
        k=self:intncmp(self.board,trial,self.size);
        if(k>0) then return 0; end
			  -- 270度回転 反対角鏡と同等
        self:rotate(trial,scratch,self.size,true);
        k=self:intncmp(self.board,trial,self.size);
        if(k>0) then return 0; end
      end 
    end
    return self.nEquiv * 2;
  end
  --
  function NQueen:NQueens(row) 
    if row==self.size then
	    local tst=self:symmetryOps();--//回転・反転・対称の解析
	    if(tst~=0) then
	      self.nUnique=self.nUnique+1;
	      self.nTotal=self.nTotal+tst;
	    end
    else
      for col=0,self.size-1,1 do
        self.board[row]=col; 
        if	self.colChk[col]==nil 
			  and self.antiChk[row+col]==nil 
			  and self.diagChk[row-col+(self.size-1)]==nil then
          self.colChk[col],
          self.antiChk[row+col],
          self.diagChk[row-col+(self.size-1)]=true,true,true;
          self:NQueens(row+1);
          self.colChk[col],
          self.antiChk[row+col],
          self.diagChk[row-col+(self.size-1)]=nil,nil,nil;
        end
      end
    end
  end
  --
  function NQueen:NQueen()
	  local max=17 ; -- //最大N
    print(" N:            Total       Unique    hh:mm:ss");
    for si=2,max,1 do
      self.size=si; --初期化
		  self.nUnique,self.nTotal=0,0;
		  self.board,self.colChk,self.diagChk,self.antiChk={},{},{},{};
      s=os.time();  --計測開始
      for k=0,self.size-1,1 do
        self.board[k]=k;
      end
      self:NQueens(0);
      print(string.format("%2d:%17d%13d%12s",
												si,self.nTotal,self.nUnique,
												self:secstotime(os.difftime(os.time(),s)))); 
    end
  end
  --
  return setmetatable( this,{__index=NQueen} );
end
--
NQueen.new():NQueen();



# 5.対称解除法＋枝刈り 

In [ ]:
#!/usr/bin/env luajit

--[[
 * Luaで学ぶアルゴリズムとデータ構造  
 * ステップバイステップでＮ−クイーン問題を最適化
 * 一般社団法人  共同通信社  情報技術局  鈴木  維一郎(suzuki.iichiro@kyodonews.jp)
 * 
 *
 * ５．対称解除法＋枝刈り
 * 　単純ですのでソースのコメントを見比べて下さい。
 *   単純ではありますが、枝刈りの効果は絶大です。
 *

 実行結果 Luajit
 N:            Total       Unique    hh:mm:ss
 2:                0            0    00:00:00
 3:                0            0    00:00:00
 4:                2            1    00:00:00
 5:               10            2    00:00:00
 6:                4            1    00:00:00
 7:               40            6    00:00:00
 8:               92           12    00:00:00
 9:              352           46    00:00:00
10:              724           92    00:00:00
11:             2680          341    00:00:00
12:            14200         1787    00:00:00
13:            73712         9233    00:00:01
14:           365596        45752    00:00:03
15:          2279184       285053    00:00:21
16:         14772512      1846955    00:02:09
17:         95815104     11977939    00:16:38
]]--

NQueen={}; NQueen.new=function()
	--
  local this={
    TOTAL=0;
    UNIQUE=0;
    colChk={};
    diagChk={};
    antiChk={};
    board={};
  };
	--
  function NQueen:secstotime(secs)
    sec=math.floor(secs);
	  if(sec>59) then
		  local hour=math.floor(sec*0.000277777778);
		  local minute=math.floor(sec*0.0166666667)-hour*60;
		  sec=sec-hour*3600-minute*60
		  if(sec<10)then sec="0"..sec; end
		  if(hour<10)then hour="0"..hour; end
		  if(minute<10)then minute="0"..minute; end
		  return hour..":"..minute..":"..sec;
	  end
	  if(sec<10)then sec="0"..sec end
	  return "00:00:"..sec;
  end 
	--
  function NQueen:intncmp(board,trial,size)
    local rtn=0; 
    for k=0,size,1 do
      rtn=board[k]-trial[k];
      if (rtn~=0)then 
        break; 
      end
    end
    return rtn;
  end
	--
  function NQueen:rotate(trial,scratch,size,neg)
    local k;
	  local incr;
    if neg then k=0; else k=size-1; end
    if neg then incr=1; else incr=-1; end 

	  local j=0;
	  while j<size do
      scratch[j]=trial[k];
		  k=k+incr;
      j=j+1;
    end

    if neg then k=size-1; else k=0; end 

	  local j=0;
	  while j<size do
      trial[scratch[j]]=k;
	    k=k-incr;
      j=j+1;
    end
  end
	--
  function NQueen:vMirror(trial,size)
    for j=0,size,1 do
      trial[j]=(size-1)-trial[j];
    end
  end
	--
  function NQueen:symmetryOps(board,trial,scratch,size)
    local nEquiv=0; 	  
    --初期化
    for i=0,size,1 do 
      trial[i]=board[i]; 
    end
    --//時計回りに90度回転
    self:rotate(trial,scratch,size,0);
    local k=self:intncmp(board,trial,size);
    if(k>0)then return 0; end
    if(k==0)then 
      nEquiv=1; 
    else
      --//時計回りに180度回転
      self:rotate(trial,scratch,size,0);
      k=self:intncmp(board,trial,size);
      if(k>0) then return 0; end
      if(k==0)then 
        nEquiv=2; 
      else
        --//時計回りに270度回転
        self:rotate(trial,scratch,size,0);
        k=self:intncmp(board,trial,size);
        if(k>0) then 
          return 0; 
        end
        nEquiv=4;
      end
    end  
    --初期化
    for i=0,size,1 do 
      trial[i]=board[i]; 
    end
    --//垂直反転 
    self:vMirror(trial,size);	
    k=self:intncmp(board,trial,size);
    if(k>0) then 
      return 0; 
    end
    --// 4回転とは異なる場合
    if (nEquiv>1) then
			-- 90度回転 対角鏡と同等
      self:rotate(trial,scratch,size,1);
      k=self:intncmp(board,trial,size);
      if(k>0) then 
        return 0; 
      end
      --// 2回転とは異なる場合
      if(nEquiv>2)then
			  -- 180度回転 水平鏡像と同等
        self:rotate(trial,scratch,size,1);
        k=self:intncmp(board,trial,size);
        if(k>0) then 
          return 0; 
        end
			  -- 270度回転 反対角鏡と同等
        self:rotate(trial,scratch,size,1);
        k=self:intncmp(board,trial,size);
        if(k>0) then 
          return 0; 
        end
      end 
    end
    return nEquiv * 2;
  end
	--
  function NQueen:NQueens(board,row,size)
    local tmp;
    local trial={};
    local scratch={};
    --枝刈り
    -- if row==size then
    if row==size-1 then
      --枝刈り
      if(self.diagChk[row-board[row]+size-1]==true
        or self.antiChk[row+board[row]]==true) then
        return;
      end
	    local tst=self:symmetryOps(board,trial,scratch,size);--//回転・反転・対称の解析
	    if(tst~=0) then
	      self.UNIQUE=self.UNIQUE+1;
        self.TOTAL=self.TOTAL+tst;
	    end
    else
      --枝刈り
      local lim ;
		  if row~=0 then lim=size; else lim=(size+1)/2; end
      --for i=0,size-1,1 do
        for i=row,lim-1,1 do
        --board[row]=i; 
          -- 交換
          tmp=board[i];
          board[i]=board[row];
          board[row]=tmp;

        if	self.antiChk[row+board[row]]==nil
			  and self.diagChk[row-board[row]+(size-1)]==nil then
          self.antiChk[row+board[row]],
          self.diagChk[row-board[row]+(size-1)]=true,true;
          self:NQueens(board,row+1,size);
          self.antiChk[row+board[row]],
          self.diagChk[row-board[row]+(size-1)]=nil,nil;
        end
      end
      -- 交換
      tmp=board[row];
      for i=row+1,size,1 do
        board[i-1]=board[i];
      end
      board[size-1]=tmp;
    end
  end
	--
  function NQueen:NQueen()
	  local max=17 ; -- //最大N
    print(" N:            Total       Unique    hh:mm:ss");
    for size=2,max,1 do
		  self.UNIQUE=0;
      self.TOTAL=0;
      s=os.time();  --計測開始
      for j=0,size,1 do
        self.board[j]=j;
      end
      self:NQueens(self.board,0,size);
      print(string.format("%2d:%17d%13d%12s",
												size,self.TOTAL,self.UNIQUE,
												self:secstotime(os.difftime(os.time(),s)))); 
    end
  end
  --
  return setmetatable( this,{__index=NQueen} );
end
--
NQueen.new():NQueen();



# 6.ビットマップ  

In [ ]:
#!/usr/bin/env luajit

--[[
  Luaで学ぶアルゴリズムとデータ構造  
  ステップバイステップでＮ−クイーン問題を最適化
  一般社団法人  共同通信社  情報技術局  鈴木  維一郎(suzuki.iichiro@kyodonews.jp)
  
  ６．バックトラック＋ビットマップ

   ビット演算を使って高速化 状態をビットマップにパックし、処理する
   単純なバックトラックよりも２０〜３０倍高速
 
 　ビットマップであれば、シフトにより高速にデータを移動できる。
  フラグ配列ではデータの移動にO(N)の時間がかかるが、ビットマップであればO(1)
  フラグ配列のように、斜め方向に 2*N-1の要素を用意するのではなく、Nビットで充
  分。

 　配置可能なビット列を flags に入れ、-flags & flags で順にビットを取り出し処理。
 　バックトラックよりも２０−３０倍高速。
 
 ===================
 考え方 1
 ===================

 　Ｎ×ＮのチェスボードをＮ個のビットフィールドで表し、ひとつの横列の状態をひと
 つのビットフィールドに対応させます。(クイーンが置いてある位置のビットをONに
 する)
 　そしてバックトラッキングは0番目のビットフィールドから「下に向かって」順にい
 ずれかのビット位置をひとつだけONにして進めていきます。

 
- - - - - Q - -    00000100 0番目のビットフィールド
- - - Q - - - -    00010000 1番目のビットフィールド
- - - - - - Q -    00000010 2番目のビットフィールド
Q - - - - - - -    10000000 3番目のビットフィールド
- - - - - - - Q    00000001 4番目のビットフィールド
- Q - - - - - -    01000000 5番目のビットフィールド
- - - - Q - - -    00001000 6番目のビットフィールド
- - Q - - - - -    00100000 7番目のビットフィールド


 ===================
 考え方 2
 ===================

 次に、効き筋をチェックするためにさらに３つのビットフィールドを用意します。

 1. 左下に効き筋が進むもの: left 
 2. 真下に効き筋が進むもの: down
 3. 右下に効き筋が進むもの: right

次に、斜めの利き筋を考えます。
 上図の場合、
 1列目の右斜め上の利き筋は 3 番目 (0x08)
 2列目の右斜め上の利き筋は 2 番目 (0x04) になります。
 この値は 0 列目のクイーンの位置 0x10 を 1 ビットずつ「右シフト」すれば求める
 ことができます。
 また、左斜め上の利き筋の場合、1 列目では 5 番目 (0x20) で 2 列目では 6 番目 (0x40)
になるので、今度は 1 ビットずつ「左シフト」すれば求めることができます。

つまり、右シフトの利き筋を right、左シフトの利き筋を left で表すことで、クイー
ンの効き筋はrightとleftを1 ビットシフトするだけで求めることができるわけです。

  *-------------
  | . . . . . .
  | . . . -3. .  0x02 -|
  | . . -2. . .  0x04  |(1 bit 右シフト right)
  | . -1. . . .  0x08 -|
  | Q . . . . .  0x10 ←(Q の位置は 4   down)
  | . +1. . . .  0x20 -| 
  | . . +2. . .  0x40  |(1 bit 左シフト left)  
  | . . . +3. .  0x80 -|
  *-------------
  図：斜めの利き筋のチェック

 n番目のビットフィールドからn+1番目のビットフィールドに探索を進めるときに、そ
 の３つのビットフィールドとn番目のビットフィールド(bit)とのOR演算をそれぞれ行
 います。leftは左にひとつシフトし、downはそのまま、rightは右にひとつシフトして
 n+1番目のビットフィールド探索に渡してやります。

 left : (left |bit)<<1
 right: (right|bit)>>1
 down :   down|bit


 ===================
 考え方 3
 ===================

   n+1番目のビットフィールドの探索では、この３つのビットフィールドをOR演算した
 ビットフィールドを作り、それがONになっている位置は効き筋に当たるので置くことが
 できない位置ということになります。次にその３つのビットフィールドをORしたビッ
 トフィールドをビット反転させます。つまり「配置可能なビットがONになったビットフィー
 ルド」に変換します。そしてこの配置可能なビットフィールドを bitmap と呼ぶとして、
 次の演算を行なってみます。
 
 bit = -bitmap & bitmap; //一番右のビットを取り出す
 
   この演算式の意味を理解するには負の値がコンピュータにおける２進法ではどのよう
 に表現されているのかを知る必要があります。負の値を２進法で具体的に表わしてみる
 と次のようになります。
 
  00000011   3
  00000010   2
  00000001   1
  00000000   0
  11111111  -1
  11111110  -2
  11111101  -3
 
   正の値nを負の値-nにするときは、nをビット反転してから+1されています。そして、
 例えばn=22としてnと-nをAND演算すると下のようになります。nを２進法で表したときの
 一番下位のONビットがひとつだけ抽出される結果が得られるのです。極めて簡単な演算
 によって1ビット抽出を実現させていることが重要です。
 
      00010110   22
  AND 11101010  -22
 ------------------
      00000010
 
   さて、そこで下のようなwhile文を書けば、このループは bitmap のONビットの数の
 回数だけループすることになります。配置可能なパターンをひとつずつ全く無駄がなく
 生成されることになります。
 
 while (bitmap) {
     bit = -bitmap & bitmap;
     bitmap ^= bit;
     //ここでは配置可能なパターンがひとつずつ生成される(bit) 
 }
 
   * 実行結果 luajit
 N:            Total       Unique    hh:mm:ss
 2:                0            0    00:00:00
 3:                0            0    00:00:00
 4:                2            0    00:00:00
 5:               10            0    00:00:00
 6:                4            0    00:00:00
 7:               40            0    00:00:00
 8:               92            0    00:00:00
 9:              352            0    00:00:00
10:              724            0    00:00:00
11:             2680            0    00:00:00
12:            14200            0    00:00:00
13:            73712            0    00:00:01
14:           365596            0    00:00:02
15:          2279184            0    00:00:11
16:         14772512            0    00:01:17
17:         95815104            0    00:09:05
]]--
--
NQueen={}; NQueen.new=function()
  -- 
  local this={
    size=0;
    TOTAL=0;
    UNIQUE=0;
    MASK=0;
  };
  --
  function NQueen:secstotime(secs)
    sec=math.floor(secs);
	  if(sec>59) then
		  local hour = math.floor(sec*0.000277777778)
		  local minute = math.floor(sec*0.0166666667) - hour*60
		  sec = sec - hour*3600 - minute*60
		  if(sec<10)then sec = "0"..sec end
		  if(hour<10)then hour = "0"..hour end
		  if(minute<10)then minute = "0"..minute end
		  return hour..":"..minute..":"..sec
	  end
	  if(sec<10)then sec = "0"..sec end
	  return "00:00:"..sec
  end 
  --
  function NQueen:NQueens(min,left,down,right) 
    local bitmap=0;
    local BIT=0;
    if min==self.size then
      self.TOTAL=self.TOTAL+1 ;
    else
      bitmap=bit.band(self.MASK,self:rbits(bit.bor(left,down,right),self.size-1));
      while bitmap~=0 do
        BIT=bit.band(-bitmap,bitmap);
        bitmap=bit.bxor(bitmap,BIT);
        self:NQueens(min+1,bit.lshift(bit.bor(left,BIT),1),bit.bor(down,BIT),bit.rshift(bit.bor(right,BIT),1));
      end
    end
  end
  --
  function NQueen:NQueen()
    local max=24;
    print(" N:            Total       Unique    hh:mm:ss");
    for si=2,max,1 do
      self.size=si;
      self.TOTAL=0;
      self.UNIQUE=0;
      self.MASK=bit.lshift(1,self.size)-1;    
      s=os.time();
      self:NQueens(0,0,0,0);
      print(string.format("%2d:%17d%13d%12s",si,self.TOTAL,0,self:secstotime(os.difftime(os.time(),s)))); 
    end
  end
  --
  return setmetatable( this,{__index=NQueen} );
end
  --ビット反転させるメソッド・・・
  function NQueen:rbits(byte,sz)
    local score=0;
    for i=sz,0,-1 do
    --io.write(bit.bnot(bit.band(bit.arshift(byte,i), 1)))
      if bit.band(bit.arshift(byte,i), 1) ==0 then
        score=score+2^i;
      end
    end
    return score;
  end
--
NQueen.new():NQueen();



# 7.ビットマップ＋対称解除法                   

In [ ]:
#!/usr/bin/env luajit

--[[
  Luaで学ぶアルゴリズムとデータ構造  
  ステップバイステップでＮ−クイーン問題を最適化
  一般社団法人  共同通信社  情報技術局  鈴木  維一郎(suzuki.iichiro@kyodonews.jp)
  
 ７．バックトラック＋ビットマップ＋対称解除法

     一つの解には、盤面を９０度、１８０度、２７０度回転、及びそれらの鏡像の合計
     ８個の対称解が存在する。対照的な解を除去し、ユニーク解から解を求める手法。
 
 ■ユニーク解の判定方法
   全探索によって得られたある１つの解が、回転・反転などによる本質的に変わること
 のない変換によって他の解と同型となるものが存在する場合、それを別の解とはしない
 とする解の数え方で得られる解を「ユニーク解」といいます。つまり、ユニーク解とは、
 全解の中から回転・反転などによる変換によって同型になるもの同士をグループ化する
 ことを意味しています。
 
   従って、ユニーク解はその「個数のみ」に着目され、この解はユニーク解であり、こ
 の解はユニーク解ではないという定まった判定方法はありません。ユニーク解であるか
 どうかの判断はユニーク解の個数を数える目的の為だけに各個人が自由に定義すること
 になります。もちろん、どのような定義をしたとしてもユニーク解の個数それ自体は変
 わりません。
 
   さて、Ｎクイーン問題は正方形のボードで形成されるので回転・反転による変換パター
 ンはぜんぶで８通りあります。だからといって「全解数＝ユニーク解数×８」と単純には
 いきません。ひとつのグループの要素数が必ず８個あるとは限らないのです。Ｎ＝５の
 下の例では要素数が２個のものと８個のものがあります。


 Ｎ＝５の全解は１０、ユニーク解は２なのです。
 
 グループ１: ユニーク解１つ目
 - - - Q -   - Q - - -
 Q - - - -   - - - - Q
 - - Q - -   - - Q - -
 - - - - Q   Q - - - -
 - Q - - -   - - - Q -
 
 グループ２: ユニーク解２つ目
 - - - - Q   Q - - - -   - - Q - -   - - Q - -   - - - Q -   - Q - - -   Q - - - -   - - - - Q
 - - Q - -   - - Q - -   Q - - - -   - - - - Q   - Q - - -   - - - Q -   - - - Q -   - Q - - -
 Q - - - -   - - - - Q   - - - Q -   - Q - - -   - - - - Q   Q - - - -   - Q - - -   - - - Q -
 - - - Q -   - Q - - -   - Q - - -   - - - Q -   - - Q - -   - - Q - -   - - - - Q   Q - - - -
 - Q - - -   - - - Q -   - - - - Q   Q - - - -   Q - - - -   - - - - Q   - - Q - -   - - Q - -

 
   それでは、ユニーク解を判定するための定義付けを行いますが、次のように定義する
 ことにします。各行のクイーンが右から何番目にあるかを調べて、最上段の行から下
 の行へ順番に列挙します。そしてそれをＮ桁の数値として見た場合に最小値になるもの
 をユニーク解として数えることにします。尚、このＮ桁の数を以後は「ユニーク判定値」
 と呼ぶことにします。
 
 - - - - Q   0
 - - Q - -   2
 Q - - - -   4   --->  0 2 4 1 3  (ユニーク判定値)
 - - - Q -   1
 - Q - - -   3
 
 
   探索によって得られたある１つの解(オリジナル)がユニーク解であるかどうかを判定
 するには「８通りの変換を試み、その中でオリジナルのユニーク判定値が最小であるか
 を調べる」ことになります。しかし結論から先にいえば、ユニーク解とは成り得ないこ
 とが明確なパターンを探索中に切り捨てるある枝刈りを組み込むことにより、３通りの
 変換を試みるだけでユニーク解の判定が可能になります。
  
 
 ■ユニーク解の個数を求める
   先ず最上段の行のクイーンの位置に着目します。その位置が左半分の領域にあればユ
 ニーク解には成り得ません。何故なら左右反転によって得られるパターンのユニーク判
 定値の方が確実に小さくなるからです。また、Ｎが奇数の場合に中央にあった場合はど
 うでしょう。これもユニーク解には成り得ません。何故なら仮に中央にあった場合、そ
 れがユニーク解であるためには少なくとも他の外側の３辺におけるクイーンの位置も中
 央になければならず、それは互いの効き筋にあたるので有り得ません。

 ■ユニーク解から全解への展開
   これまでの考察はユニーク解の個数を求めるためのものでした。全解数を求めるには
 ユニーク解を求めるための枝刈りを取り除いて全探索する必要があります。したがって
 探索時間を犠牲にしてしまうことになります。そこで「ユニーク解の個数から全解数を
 導いてしまおう」という試みが考えられます。これは、左右反転によるパターンの探索
 を省略して最後に結果を２倍するというアイデアの拡張版といえるものです。そしてそ
 れを実現させるには「あるユニーク解が属するグループの要素数はいくつあるのか」と
 いう考察が必要になってきます。
 
   最初に、クイーンが右上角にあるユニーク解を考えます。斜軸で反転したパターンが
 オリジナルと同型になることは有り得ないことと(×２)、右上角のクイーンを他の３つの
 角に写像させることができるので(×４)、このユニーク解が属するグループの要素数は必
 ず８個(＝２×４)になります。
 
   次に、クイーンが右上角以外にある場合は少し複雑になりますが、考察を簡潔にする
 ために次の事柄を確認します。

 TOTAL = (COUNT8 * 8) + (COUNT4 * 4) + (COUNT2 * 2);
   (1) 90度回転させてオリジナルと同型になる場合、さらに90度回転(オリジナルか
    ら180度回転)させても、さらに90度回転(オリジナルから270度回転)させてもオリ
    ジナルと同型になる。  

    COUNT2 * 2
 
   (2) 90度回転させてオリジナルと異なる場合は、270度回転させても必ずオリジナ
    ルとは異なる。ただし、180度回転させた場合はオリジナルと同型になることも有
    り得る。 

    COUNT4 * 4
 
   (3) (1) に該当するユニーク解が属するグループの要素数は、左右反転させたパターンを
       加えて２個しかありません。(2)に該当するユニーク解が属するグループの要素数は、
       180度回転させて同型になる場合は４個(左右反転×縦横回転)、そして180度回転させても
       オリジナルと異なる場合は８個になります。(左右反転×縦横回転×上下反転)
 
    COUNT8 * 8 

   以上のことから、ひとつひとつのユニーク解が上のどの種類に該当するのかを調べる
 ことにより全解数を計算で導き出すことができます。探索時間を短縮させてくれる枝刈
 りを外す必要がなくなったというわけです。 
 
   UNIQUE  COUNT2      +  COUNT4      +  COUNT8
   TOTAL  (COUNT2 * 2) + (COUNT4 * 4) + (COUNT8 * 8)

 　これらを実現すると、前回のNQueen3()よりも実行速度が遅くなります。
 　なぜなら、対称・反転・斜軸を反転するための処理が加わっているからです。
 ですが、今回の処理を行うことによって、さらに、処理スピードが飛躍的に高速化されます。そのためにも今回のアルゴリズム実装は必要なのです。



	実行結果
 N:            Total       Unique    hh:mm:ss
 2:                0            0    00:00:00
 3:                0            0    00:00:00
 4:                2            1    00:00:00
 5:               10            2    00:00:00
 6:                4            1    00:00:00
 7:               40            6    00:00:00
 8:               92           12    00:00:00
 9:              352           46    00:00:00
10:              724           92    00:00:00
11:             2680          341    00:00:00
12:            14200         1787    00:00:00
13:            73712         9233    00:00:00
14:           365596        45752    00:00:05
15:          2279184       285053    00:00:27
16:         14772512      1846955    00:03:12
17:         95815104     11977939    00:23:09

 ]]--


NQueen={}; NQueen.new=function()
  -- 
  local this={
    size=0;
    TOTAL=0;
    UNIQUE=0;
    MASK=0;
    --nTotal=0;nUniq=0;nEquiv=0; 
    COUNT2=0;COUNT4=0;COUNT8=0;
    bit;
    board={};trial={};scratch={};
    -- trial={};scratch={};
  };
  --
  function NQueen:secstotime(secs)
    sec=math.floor(secs);
	  if(sec>59) then
		  local hour = math.floor(sec*0.000277777778)
		  local minute = math.floor(sec*0.0166666667) - hour*60
		  sec = sec - hour*3600 - minute*60
		  if(sec<10)then sec = "0"..sec end
		  if(hour<10)then hour = "0"..hour end
		  if(minute<10)then minute = "0"..minute end
		  return hour..":"..minute..":"..sec
	  end
	  if(sec<10)then sec = "0"..sec end
	  return "00:00:"..sec
  end 
  --
  function NQueen:rotate(bf,af,si)
    for i=0,si,1 do
      local t=0;
      for j=0,si,1 do
        --t=t|((bf[j]>>i)&1)<<(si-j-1); 
        t=bit.bor(t,bit.lshift(bit.band(bit.rshift(bf[j],i),1),(si-j-1)));
      end 
      af[i]=t;
    end 
  end
  --
  function NQueen:vMirror(bf,af,si)
    local score ;
    for i=0,si,1 do 
      score=bf[i];
      af[i]=self:rh(score,si-1);
    end 
  end
  --
  function NQueen:rh(a,sz)
    local tmp=0;
    for i=0,sz,1 do
      --if(a&(1<<i))then  
      if bit.band(a,bit.lshift(1,i))~=0 then
         --return tmp|=(1<<(sz-i)); 
         return bit.bor(tmp,bit.lshift(1,(sz-i)));
      end
    end 
    return tmp;
  end
  --
  function NQueen:intncmp(lt,rt,si)
    local rtn=0;
    for k=0,si,1 do
      rtn=lt[k]-rt[k];
      if(rtn~=0)then break;end
    end 
    return rtn;
  end
  --
  function NQueen:symmetryOps(si)
    local nEquiv;
    --// 回転・反転・対称チェックのためにboard配列をコピー
    for i=0,si,1 do self.trial[i]=self.board[i]; end
    --//時計回りに90度回転
    self:rotate(self.trial,self.scratch,si);    
    local k=self:intncmp(self.board,self.scratch,si);
    if(k>0)then return; end
    if(k==0)then nEquiv=2;
    else
      --//時計回りに180度回転
      self:rotate(self.scratch,self.trial,si);  
      k=self:intncmp(self.board,self.trial,si);
      if(k>0)then return; end 
      if(k==0)then nEquiv=4;
      else
        --//時計回りに270度回転
        self:rotate(self.trial,self.scratch,si);
        k=self:intncmp(self.board,self.scratch,si);
        if(k>0) then return; end 
        nEquiv=8;
      end 
    end 
    --// 回転・反転・対称チェックのためにboard配列をコピー
    for i=0,si,1 do
      self.scratch[i]=self.board[i];
    end
    --//垂直反転
    self:vMirror(self.scratch,self.trial,si);   
    k=self:intncmp(self.board,self.trial,si);
    if(k>0)then return; end 
    if(nEquiv>2)then    --//-90度回転 対角鏡と同等       
      self:rotate(self.trial,self.scratch,si);
      k=self:intncmp(self.board,self.scratch,si);
      if(k>0)then return; end 
      if(nEquiv>4)then  --//-180度回転 水平鏡像と同等
        self:rotate(self.scratch,self.trial,si);
        k=self:intncmp(self.board,self.trial,si);
        if(k>0)then return; end
        --//-270度回転 反対角鏡と同等
        self:rotate(self.trial,self.scratch,si);
        k=self:intncmp(self.board,self.scratch,si);
        if(k>0)then return; end 
      end 
    end 
    if(nEquiv==2)then self.COUNT2=self.COUNT2+1;end 
    if(nEquiv==4)then self.COUNT4=self.COUNT4+1;end 
    if(nEquiv==8)then self.COUNT8=self.COUNT8+1;end
  end
  --
  function NQueen:NQueens(min,left,down,right) 
    local bitmap=bit.band(self.MASK,self:rbits(bit.bor(left,down,right ),self.size-1));
    if min==self.size then
      --self.TOTAL=self.TOTAL+1 ;
      if bitmap==0 then
        self.board[min]=bitmap;
        self:symmetryOps(self.size);
      end
    else
      while bitmap~=0 do
        self.BIT=bit.band(-bitmap,bitmap);
        self.board[min]=self.BIT;
        bitmap=bit.bxor(bitmap,self.BIT);
        self:NQueens(min+1,bit.lshift(bit.bor(left,self.BIT),1),bit.bor(down,self.BIT),bit.rshift(bit.bor(right,self.BIT),1));
      end
    end
  end
  --
  function NQueen:rbits(byte,sz)
    local score=0;
    for i=sz,0,-1 do
      if bit.band(bit.arshift(byte,i), 1) ==0 then
        score=score+2^i;
      end
    end
    return score;
  end
  --
  function NQueen:NQueen()
    local max=17;
    print(" N:            Total       Unique    hh:mm:ss");
    for si=2,max,1 do
      self.size=si;
      self.TOTAL=0;
      self.UNIQUE=0;
      self.COUNT2=0;
      self.COUNT4=0;
      self.COUNT8=0;
      for k=0,self.size-1,1 do self.board[k]=k; end --テーブルの初期化
      self.MASK=bit.lshift(1,self.size)-1;    
      s=os.time();
      self:NQueens(0,0,0,0);
      self.TOTAL=self.COUNT2*2+self.COUNT4*4+self.COUNT8*8;
      self.UNIQUE=self.COUNT2+self.COUNT4+self.COUNT8;
      print(string.format("%2d:%17d%13d%12s",si,self.TOTAL,self.UNIQUE,self:secstotime(os.difftime(os.time(),s)))); 
    end
  end
  return setmetatable( this,{__index=NQueen} );
end
  --
NQueen.new():NQueen();


# 8.ビットマップ＋対称解除法＋枝刈り 

In [ ]:
#!/usr/bin/env luajit

--[[
  Luaで学ぶアルゴリズムとデータ構造  
  ステップバイステップでＮ−クイーン問題を最適化
  一般社団法人  共同通信社  情報技術局  鈴木  維一郎(suzuki.iichiro@kyodonews.jp)
  
 ８．バックトラック＋ビットマップ＋対称解除法＋枝刈りと最適化

 	実行結果

 N:            Total       Unique    hh:mm:ss
 2:                0            0    00:00:00
 3:                0            0    00:00:00
 4:                2            1    00:00:00
 5:               10            2    00:00:00
 6:                4            1    00:00:00
 7:               40            6    00:00:00
 8:               92           12    00:00:00
 9:              352           46    00:00:00
10:              724           92    00:00:00
11:             2680          341    00:00:00
12:            14200         1787    00:00:00
13:            73712         9233    00:00:01
14:           365596        45752    00:00:01
15:          2279184       285053    00:00:10
16:         14772512      1846955    00:01:01
17:         95815104     11977939    00:07:33

  ]]--

NQueen={}; NQueen.new=function()
  -- 
  local this={
    size=0;
    TOTAL=0;
    UNIQUE=0;
    MASK=0;
    --nTotal=0;nUniq=0;nEquiv=0; 
    COUNT2=0;COUNT4=0;COUNT8=0;
    bit;
    board={};trial={};scratch={};
    -- trial={};scratch={};
  };
  --
  function NQueen:secstotime(secs)
    sec=math.floor(secs);
	  if(sec>59) then
		  local hour = math.floor(sec*0.000277777778)
		  local minute = math.floor(sec*0.0166666667) - hour*60
		  sec = sec - hour*3600 - minute*60
		  if(sec<10)then sec = "0"..sec end
		  if(hour<10)then hour = "0"..hour end
		  if(minute<10)then minute = "0"..minute end
		  return hour..":"..minute..":"..sec
	  end
	  if(sec<10)then sec = "0"..sec end
	  return "00:00:"..sec
  end 
  --
  function NQueen:rotate(bf,af,si)
    for i=0,si,1 do
      local t=0;
      for j=0,si,1 do
        --t=t|((bf[j]>>i)&1)<<(si-j-1); 
        t=bit.bor(t,bit.lshift(bit.band(bit.rshift(bf[j],i),1),(si-j-1)));
      end 
      af[i]=t;
    end 
  end
  --
  function NQueen:vMirror(bf,af,si)
    local score ;
    for i=0,si,1 do 
      score=bf[i];
      af[i]=self:rh(score,si-1);
    end 
  end
  --
  function NQueen:rh(a,sz)
    local tmp=0;
    for i=0,sz,1 do
      --if(a&(1<<i))then  
      if bit.band(a,bit.lshift(1,i))~=0 then
         --return tmp|=(1<<(sz-i)); 
         return bit.bor(tmp,bit.lshift(1,(sz-i)));
      end
    end 
    return tmp;
  end
  --
  function NQueen:intncmp(lt,rt,si)
    local rtn=0;
    for k=0,si,1 do
      rtn=lt[k]-rt[k];
      if(rtn~=0)then break;end
    end 
    return rtn;
  end
  --
  function NQueen:symmetryOps(si)
    local nEquiv;
    --// 回転・反転・対称チェックのためにboard配列をコピー
    for i=0,si,1 do self.trial[i]=self.board[i]; end
    --//時計回りに90度回転
    self:rotate(self.trial,self.scratch,si);    
    local k=self:intncmp(self.board,self.scratch,si);
    if(k>0)then return; end
    if(k==0)then nEquiv=2;
    else
      --//時計回りに180度回転
      self:rotate(self.scratch,self.trial,si);  
      k=self:intncmp(self.board,self.trial,si);
      if(k>0)then return; end 
      if(k==0)then nEquiv=4;
      else
        --//時計回りに270度回転
        self:rotate(self.trial,self.scratch,si);
        k=self:intncmp(self.board,self.scratch,si);
        if(k>0) then return; end 
        nEquiv=8;
      end 
    end 
    --// 回転・反転・対称チェックのためにboard配列をコピー
    for i=0,si,1 do
      self.scratch[i]=self.board[i];
    end
    --//垂直反転
    self:vMirror(self.scratch,self.trial,si);   
    k=self:intncmp(self.board,self.trial,si);
    if(k>0)then return; end 
    if(nEquiv>2)then    --//-90度回転 対角鏡と同等       
      self:rotate(self.trial,self.scratch,si);
      k=self:intncmp(self.board,self.scratch,si);
      if(k>0)then return; end 
      if(nEquiv>4)then  --//-180度回転 水平鏡像と同等
        self:rotate(self.scratch,self.trial,si);
        k=self:intncmp(self.board,self.trial,si);
        if(k>0)then return; end
        --//-270度回転 反対角鏡と同等
        self:rotate(self.trial,self.scratch,si);
        k=self:intncmp(self.board,self.scratch,si);
        if(k>0)then return; end 
      end 
    end 
    if(nEquiv==2)then self.COUNT2=self.COUNT2+1;end 
    if(nEquiv==4)then self.COUNT4=self.COUNT4+1;end 
    if(nEquiv==8)then self.COUNT8=self.COUNT8+1;end
  end
  --
  function NQueen:NQueens(min,left,down,right) 
    local bitmap=bit.band(self.MASK,self:rbits(bit.bor(left,down,right ),self.size-1));
    if min==self.size then
      --self.TOTAL=self.TOTAL+1 ;
      if bitmap==0 then
        self.board[min]=bitmap;
        self:symmetryOps(self.size);
      end
    else
      if min~=0 then
        lim=self.size;
      else
        -- 枝刈り
        lim=(self.size+1)/2;
      end
      -- 枝刈り
      for s=min+1,lim,1 do
        if bitmap==0 then
          break;
        end
      --while bitmap~=0 do
        self.BIT=bit.band(-bitmap,bitmap);
        self.board[min]=self.BIT;
        bitmap=bit.bxor(bitmap,self.BIT);
        self:NQueens(min+1,bit.lshift(bit.bor(left,self.BIT),1),bit.bor(down,self.BIT),bit.rshift(bit.bor(right,self.BIT),1));
      end
    end
  end
  --
  function NQueen:rbits(byte,sz)
    local score=0;
    for i=sz,0,-1 do
      if bit.band(bit.arshift(byte,i), 1) ==0 then
        score=score+2^i;
      end
    end
    return score;
  end
  --
  function NQueen:NQueen()
    local max=17;
    print(" N:            Total       Unique    hh:mm:ss");
    for si=2,max,1 do
      self.size=si;
      self.TOTAL=0;
      self.UNIQUE=0;
      self.COUNT2=0;
      self.COUNT4=0;
      self.COUNT8=0;
      for k=0,self.size-1,1 do self.board[k]=k; end --テーブルの初期化
      self.MASK=bit.lshift(1,self.size)-1;    
      s=os.time();
      self:NQueens(0,0,0,0);
      self.TOTAL=self.COUNT2*2+self.COUNT4*4+self.COUNT8*8;
      self.UNIQUE=self.COUNT2+self.COUNT4+self.COUNT8;
      print(string.format("%2d:%17d%13d%12s",si,self.TOTAL,self.UNIQUE,self:secstotime(os.difftime(os.time(),s)))); 
    end
  end
  return setmetatable( this,{__index=NQueen} );
end
  --
NQueen.new():NQueen();


# 9.クイーンの位置による振り分け(BOUND1)

In [ ]:
#!/usr/bin/env luajit

--[[
  Luaで学ぶアルゴリズムとデータ構造  
  ステップバイステップでＮ−クイーン問題を最適化
  一般社団法人  共同通信社  情報技術局  鈴木  維一郎(suzuki.iichiro@kyodonews.jp)
  
 ９．クイーンの位置による振り分け(BOUND1) 

 	実行結果

 N:            Total       Unique    hh:mm:ss
 2:                0            0    00:00:00
 3:                0            0    00:00:00
 4:                2            1    00:00:00
 5:               10            2    00:00:00
 6:                4            1    00:00:00
 7:               40            6    00:00:00
 8:               92           12    00:00:00
 9:              352           46    00:00:00
10:              724           92    00:00:00
11:             2680          341    00:00:00
12:            14200         1787    00:00:00
13:            73712         9233    00:00:00
14:           365596        45752    00:00:03
15:          2279184       285053    00:00:15
16:         14772512      1846955    00:01:44
17:         95815104     11977939    00:12:20

  ]]--

NQueen={}; NQueen.new=function()
  -- 
  local this={
    size=0;
    TOTAL=0;
    UNIQUE=0;
    MASK=0;
    COUNT2=0;COUNT4=0;COUNT8=0;
    BOUND1=0;
    BOUND2=0;
    TOPBIT=0;
    ENDBIT=0;
    SIDEMASK=0;
    LASTMASK=0;
    board={};trial={};scratch={};
  };
  --
  function NQueen:secstotime(secs)
    sec=math.floor(secs);
	  if(sec>59) then
		  local hour = math.floor(sec*0.000277777778)
		  local minute = math.floor(sec*0.0166666667) - hour*60
		  sec = sec - hour*3600 - minute*60
		  if(sec<10)then sec = "0"..sec end
		  if(hour<10)then hour = "0"..hour end
		  if(minute<10)then minute = "0"..minute end
		  return hour..":"..minute..":"..sec
	  end
	  if(sec<10)then sec = "0"..sec end
	  return "00:00:"..sec
  end 
  --
  function NQueen:rotate(bf,af,si)
    for i=0,si,1 do
      local t=0;
      for j=0,si,1 do
        --t=t|((bf[j]>>i)&1)<<(si-j-1); 
        t=bit.bor(t,bit.lshift(bit.band(bit.rshift(bf[j],i),1),(si-j-1)));
      end 
      af[i]=t;
    end 
  end
  --
  function NQueen:rh(a,sz)
    local tmp=0;
    for i=0,sz,1 do
      --if(a&(1<<i))then  
      if bit.band(a,bit.lshift(1,i))~=0 then
         --return tmp|=(1<<(sz-i)); 
         return bit.bor(tmp,bit.lshift(1,(sz-i)));
      end
    end 
    return tmp;
  end
  --
  function NQueen:vMirror(bf,af,si)
    local score ;
    for i=0,si,1 do 
      score=bf[i];
      af[i]=self:rh(score,si-1);
    end 
  end
  --
  function NQueen:intncmp(lt,rt,si)
    local rtn=0;
    for k=0,si,1 do
      rtn=lt[k]-rt[k];
      if(rtn~=0)then break;end
    end 
    return rtn;
  end
  --
  function NQueen:rbits(byte,sz)
    local score=0;
    for i=sz,0,-1 do
      if bit.band(bit.arshift(byte,i), 1) ==0 then
        score=score+2^i;
      end
    end
    return score;
  end
  --
  function NQueen:symmetryOps(si)
    local nEquiv;
    --// 回転・反転・対称チェックのためにboard配列をコピー
    for i=0,si,1 do self.trial[i]=self.board[i]; end
    --//時計回りに90度回転
    self:rotate(self.trial,self.scratch,si);    
    local k=self:intncmp(self.board,self.scratch,si);
    if(k>0)then return; end
    if(k==0)then nEquiv=2;
    else
      --//時計回りに180度回転
      self:rotate(self.scratch,self.trial,si);  
      k=self:intncmp(self.board,self.trial,si);
      if(k>0)then return; end 
      if(k==0)then nEquiv=4;
      else
        --//時計回りに270度回転
        self:rotate(self.trial,self.scratch,si);
        k=self:intncmp(self.board,self.scratch,si);
        if(k>0) then return; end 
        nEquiv=8;
      end 
    end 
    --// 回転・反転・対称チェックのためにboard配列をコピー
    for i=0,si,1 do
      self.scratch[i]=self.board[i];
    end
    --//垂直反転
    self:vMirror(self.scratch,self.trial,si);   
    k=self:intncmp(self.board,self.trial,si);
    if(k>0)then return; end 
    if(nEquiv>2)then    --//-90度回転 対角鏡と同等       
      self:rotate(self.trial,self.scratch,si);
      k=self:intncmp(self.board,self.scratch,si);
      if(k>0)then return; end 
      if(nEquiv>4)then  --//-180度回転 水平鏡像と同等
        self:rotate(self.scratch,self.trial,si);
        k=self:intncmp(self.board,self.trial,si);
        if(k>0)then return; end
        --//-270度回転 反対角鏡と同等
        self:rotate(self.trial,self.scratch,si);
        k=self:intncmp(self.board,self.scratch,si);
        if(k>0)then return; end 
      end 
    end 
    if(nEquiv==2)then self.COUNT2=self.COUNT2+1;end 
    if(nEquiv==4)then self.COUNT4=self.COUNT4+1;end 
    if(nEquiv==8)then self.COUNT8=self.COUNT8+1;end
  end
  --
  function NQueen:backTrack(min,left,down,right) 
    local BIT;
    local bitmap=bit.band(self.MASK,self:rbits(bit.bor(left,down,right ),self.size-1));
    if min==self.size then
      --self.TOTAL=self.TOTAL+1 ;
      if bitmap==0 then
        self.board[min]=bitmap;
        self:symmetryOps(self.size);
      end
    else
      while bitmap~=0 do
        BIT=bit.band(-bitmap,bitmap);
        self.board[min]=BIT;
        bitmap=bit.bxor(bitmap,BIT);
        self:backTrack(min+1,bit.lshift(bit.bor(left,BIT),1),bit.bor(down,BIT),bit.rshift(bit.bor(right,BIT),1));
      end
    end
  end
  --
  function NQueen:NQueens(min) 
    local BIT;
    self.TOPBIT=bit.lshift(self.TOPBIT,(self.size-1));
    self.LASTMASK=bit.bor(self.TOPBIT,1);
    self.SIDEMASK=self.LASTMASK;
    self.ENDBIT=bit.rshift(self.TOPBIT,1);
    self.BOUND2=self.size-2;
    for BOUND1=0,self.BOUND2,1 do
      BIT=bit.lshift(1,BOUND1);
      self.board[0]=BIT;
      self:backTrack(1,bit.lshift(BIT,1),BIT,bit.rshift(BIT,1));
      self.LASTMASK=bit.bxor(self.LASTMASK,bit.bor(bit.rshift(self.LASTMASK,1),bit.lshift(self.LASTMASK,1)));
      self.ENDBIT=bit.rshift(self.ENDBIT,1);
      self.BOUND2=self.BOUND2-1;
    end
  end
  --
  function NQueen:NQueen()
    local max=17;
    print(" N:            Total       Unique    hh:mm:ss");
    for si=2,max,1 do
      self.size=si;
      self.TOTAL=0;
      self.UNIQUE=0;
      self.COUNT2=0;
      self.COUNT4=0;
      self.COUNT8=0;
      for k=0,self.size-1,1 do self.board[k]=k; end --テーブルの初期化
      self.MASK=bit.lshift(1,self.size)-1;    
      s=os.time();
      self:NQueens(0);
      self.TOTAL=self.COUNT2*2+self.COUNT4*4+self.COUNT8*8;
      self.UNIQUE=self.COUNT2+self.COUNT4+self.COUNT8;
      print(string.format("%2d:%17d%13d%12s",si,self.TOTAL,self.UNIQUE,self:secstotime(os.difftime(os.time(),s)))); 
    end
  end
  return setmetatable( this,{__index=NQueen} );
end
  --
NQueen.new():NQueen();


# 10.クイーンの位置による振り分け(BOUND1,BOUND2)

In [ ]:
#!/usr/bin/env luajit

--[[
  Luaで学ぶアルゴリズムとデータ構造  
  ステップバイステップでＮ−クイーン問題を最適化
  一般社団法人  共同通信社  情報技術局  鈴木  維一郎(suzuki.iichiro@kyodonews.jp)
  
 １０．バックトラック＋ビットマップ＋対称解除法＋枝刈りと最適化＋対称解除法のビットマップ化＋クイーンの位置による振り分け（BOUND1+BOUND2)

 	実行結果

 N:            Total       Unique    hh:mm:ss
 2:                0            0    00:00:00
 3:                0            0    00:00:00
 4:                2            1    00:00:00
 5:               10            2    00:00:00
 6:                4            1    00:00:00
 7:               40            6    00:00:00
 8:               92           12    00:00:00
 9:              352           46    00:00:00
10:              724           92    00:00:00
11:             2680          341    00:00:00
12:            14200         1787    00:00:00
13:            73712         9233    00:00:00
14:           365596        45752    00:00:02
15:          2279184       285053    00:00:16
16:         14772512      1846955    00:01:45
17:         95815104     11977939    00:12:18

  ]]--

NQueen={}; NQueen.new=function()
  -- 
  local this={
    size=0;
    UNIQUE=0;
    MASK=0;
    COUNT2=0;COUNT4=0;COUNT8=0;
    BOUND1=0;
    BOUND2=0;
    TOPBIT=0;
    ENDBIT=0;
    SIDEMASK=0;
    LASTMASK=0;
    board={};trial={};scratch={};
  };
  --
  function NQueen:secstotime(secs)
    sec=math.floor(secs);
	  if(sec>59) then
		  local hour = math.floor(sec*0.000277777778)
		  local minute = math.floor(sec*0.0166666667) - hour*60
		  sec = sec - hour*3600 - minute*60
		  if(sec<10)then sec = "0"..sec end
		  if(hour<10)then hour = "0"..hour end
		  if(minute<10)then minute = "0"..minute end
		  return hour..":"..minute..":"..sec
	  end
	  if(sec<10)then sec = "0"..sec end
	  return "00:00:"..sec
  end 
  --
  function NQueen:rotate(bf,af,si)
    for i=0,si,1 do
      local t=0;
      for j=0,si,1 do
        --t=t|((bf[j]>>i)&1)<<(si-j-1); 
        t=bit.bor(t,bit.lshift(bit.band(bit.rshift(bf[j],i),1),(si-j-1)));
      end 
      af[i]=t;
    end 
  end
  --
  function NQueen:rh(a,sz)
    local tmp=0;
    for i=0,sz,1 do
      --if(a&(1<<i))then  
      if bit.band(a,bit.lshift(1,i))~=0 then
         --return tmp|=(1<<(sz-i)); 
         return bit.bor(tmp,bit.lshift(1,(sz-i)));
      end
    end 
    return tmp;
  end
  --
  function NQueen:vMirror(bf,af,si)
    local score ;
    for i=0,si,1 do 
      score=bf[i];
      af[i]=self:rh(score,si-1);
    end 
  end
  --
  function NQueen:intncmp(lt,rt,si)
    local rtn=0;
    for k=0,si,1 do
      rtn=lt[k]-rt[k];
      if(rtn~=0)then break;end
    end 
    return rtn;
  end
  --
  function NQueen:rbits(byte,sz)
    local score=0;
    for i=sz,0,-1 do
      if bit.band(bit.arshift(byte,i), 1) ==0 then
        score=score+2^i;
      end
    end
    return score;
  end
  --
  function NQueen:symmetryOps(si)
    local nEquiv;
    --// 回転・反転・対称チェックのためにboard配列をコピー
    for i=0,si,1 do self.trial[i]=self.board[i]; end
    --//時計回りに90度回転
    self:rotate(self.trial,self.scratch,si);    
    local k=self:intncmp(self.board,self.scratch,si);
    if(k>0)then return; end
    if(k==0)then nEquiv=2;
    else
      --//時計回りに180度回転
      self:rotate(self.scratch,self.trial,si);  
      k=self:intncmp(self.board,self.trial,si);
      if(k>0)then return; end 
      if(k==0)then nEquiv=4;
      else
        --//時計回りに270度回転
        self:rotate(self.trial,self.scratch,si);
        k=self:intncmp(self.board,self.scratch,si);
        if(k>0) then return; end 
        nEquiv=8;
      end 
    end 
    --// 回転・反転・対称チェックのためにboard配列をコピー
    for i=0,si,1 do
      self.scratch[i]=self.board[i];
    end
    --//垂直反転
    self:vMirror(self.scratch,self.trial,si);   
    k=self:intncmp(self.board,self.trial,si);
    if(k>0)then return; end 
    if(nEquiv>2)then    --//-90度回転 対角鏡と同等       
      self:rotate(self.trial,self.scratch,si);
      k=self:intncmp(self.board,self.scratch,si);
      if(k>0)then return; end 
      if(nEquiv>4)then  --//-180度回転 水平鏡像と同等
        self:rotate(self.scratch,self.trial,si);
        k=self:intncmp(self.board,self.trial,si);
        if(k>0)then return; end
        --//-270度回転 反対角鏡と同等
        self:rotate(self.trial,self.scratch,si);
        k=self:intncmp(self.board,self.scratch,si);
        if(k>0)then return; end 
      end 
    end 
    if(nEquiv==2)then self.COUNT2=self.COUNT2+1;end 
    if(nEquiv==4)then self.COUNT4=self.COUNT4+1;end 
    if(nEquiv==8)then self.COUNT8=self.COUNT8+1;end
  end
  --
  function NQueen:backTrack2(min,left,down,right) 
    local BIT;
    local bitmap=bit.band(self.MASK,self:rbits(bit.bor(left,down,right ),self.size-1));
    if min==self.size then
      --self.TOTAL=self.TOTAL+1 ;
      if bitmap==0 then
        self.board[min]=bitmap;
        self:symmetryOps(self.size);
      end
    else
      while bitmap~=0 do
        BIT=bit.band(-bitmap,bitmap);
        self.board[min]=BIT;
        bitmap=bit.bxor(bitmap,BIT);
        self:backTrack2(min+1,bit.lshift(bit.bor(left,BIT),1),bit.bor(down,BIT),bit.rshift(bit.bor(right,BIT),1));
      end
    end
  end
  --
  function NQueen:backTrack1(min,left,down,right) 
    local BIT;
    local bitmap=bit.band(self.MASK,self:rbits(bit.bor(left,down,right ),self.size-1));
    if min==self.size then
      --self.TOTAL=self.TOTAL+1 ;
      if bitmap==0 then
        self.board[min]=bitmap;
        self:symmetryOps(self.size);
      end
    else
      while bitmap~=0 do
        BIT=bit.band(-bitmap,bitmap);
        self.board[min]=BIT;
        bitmap=bit.bxor(bitmap,BIT);
        self:backTrack1(min+1,bit.lshift(bit.bor(left,BIT),1),bit.bor(down,BIT),bit.rshift(bit.bor(right,BIT),1));
      end
    end
  end
  --
  function NQueen:NQueens(min) 
    local BIT;
    self.TOPBIT=bit.lshift(1,(self.size-1));
    self.board[0]=1;
    for BOUND1=2,self.size-1,1 do
      BIT=bit.lshift(1,BOUND1);
      self.board[1]=BIT;
      self:backTrack1(2,bit.lshift(bit.bor(2,BIT),1),bit.bor(1,BIT),bit.rshift(BIT,1));
    end
    self.LASTMASK=bit.bor(self.TOPBIT,1);
    self.SIDEMASK=self.LASTMASK;
    self.ENDBIT=bit.rshift(self.TOPBIT,1);
    self.BOUND2=self.size-2;
    for BOUND1=1,self.BOUND2,1 do
      BIT=bit.lshift(1,BOUND1);
      self.board[0]=BIT;
      self:backTrack2(1,bit.lshift(BIT,1),BIT,bit.rshift(BIT,1));
      self.LASTMASK=bit.bxor(self.LASTMASK,bit.bor(bit.rshift(self.LASTMASK,1),bit.lshift(self.LASTMASK,1)));
      self.ENDBIT=bit.rshift(self.ENDBIT,1);
      self.BOUND2=self.BOUND2-1;
    end
  end
  --
  function NQueen:NQueen()
    local max=17;
    print(" N:            Total       Unique    hh:mm:ss");
    for si=2,max,1 do
      self.size=si;
      self.TOTAL=0;
      self.UNIQUE=0;
      self.COUNT2=0;
      self.COUNT4=0;
      self.COUNT8=0;
      for k=0,self.size-1,1 do self.board[k]=k; end --テーブルの初期化
      self.MASK=bit.lshift(1,self.size)-1;    
      s=os.time();
      self:NQueens(0);
      self.TOTAL=self.COUNT2*2+self.COUNT4*4+self.COUNT8*8;
      self.UNIQUE=self.COUNT2+self.COUNT4+self.COUNT8;
      print(string.format("%2d:%17d%13d%12s",si,self.TOTAL,self.UNIQUE,self:secstotime(os.difftime(os.time(),s)))); 
    end
  end
  return setmetatable( this,{__index=NQueen} );
end
  --
NQueen.new():NQueen();


# 11.枝刈り 

In [ ]:
#!/usr/bin/env luajit

--[[
  Luaで学ぶアルゴリズムとデータ構造  
  ステップバイステップでＮ−クイーン問題を最適化
  一般社団法人  共同通信社  情報技術局  鈴木  維一郎(suzuki.iichiro@kyodonews.jp)
  
 １１．バックトラック＋ビットマップ＋対称解除法＋枝刈りと最適化＋対称解除法のビッ トマップ化＋クイーンの位置による振り分け（BOUND1+BOUND2)＋枝刈り

 	実行結果

 N:            Total       Unique    hh:mm:ss
 2:                0            0    00:00:00
 3:                0            0    00:00:00
 4:                2            1    00:00:00
 5:               10            2    00:00:00
 6:                4            1    00:00:00
 7:               40            6    00:00:00
 8:               92           12    00:00:00
 9:              352           46    00:00:00
10:              724           92    00:00:00
11:             2680          341    00:00:00
12:            14200         1787    00:00:00
13:            73712         9233    00:00:00
14:           365596        45752    00:00:01
15:          2279184       285053    00:00:04
16:         14772512      1846955    00:00:30
17:         95815104     11977939    00:03:32

  ]]--

NQueen={}; NQueen.new=function()
  -- 
  local this={
    size=0;
    UNIQUE=0;
    MASK=0;
    COUNT2=0;COUNT4=0;COUNT8=0;
    BOUND1=0;
    BOUND2=0;
    TOPBIT=0;
    ENDBIT=0;
    SIDEMASK=0;
    LASTMASK=0;
    board={};trial={};scratch={};
  };
  --
  function NQueen:secstotime(secs)
    sec=math.floor(secs);
	  if(sec>59) then
		  local hour = math.floor(sec*0.000277777778)
		  local minute = math.floor(sec*0.0166666667) - hour*60
		  sec = sec - hour*3600 - minute*60
		  if(sec<10)then sec = "0"..sec end
		  if(hour<10)then hour = "0"..hour end
		  if(minute<10)then minute = "0"..minute end
		  return hour..":"..minute..":"..sec
	  end
	  if(sec<10)then sec = "0"..sec end
	  return "00:00:"..sec
  end 
  --
  function NQueen:rotate(bf,af,si)
    for i=0,si,1 do
      local t=0;
      for j=0,si,1 do
        --t=t|((bf[j]>>i)&1)<<(si-j-1); 
        t=bit.bor(t,bit.lshift(bit.band(bit.rshift(bf[j],i),1),(si-j-1)));
      end 
      af[i]=t;
    end 
  end
  --
  function NQueen:rh(a,sz)
    local tmp=0;
    for i=0,sz,1 do
      --if(a&(1<<i))then  
      if bit.band(a,bit.lshift(1,i))~=0 then
         --return tmp|=(1<<(sz-i)); 
         return bit.bor(tmp,bit.lshift(1,(sz-i)));
      end
    end 
    return tmp;
  end
  --
  function NQueen:vMirror(bf,af,si)
    local score ;
    for i=0,si,1 do 
      score=bf[i];
      af[i]=self:rh(score,si-1);
    end 
  end
  --
  function NQueen:intncmp(lt,rt,si)
    local rtn=0;
    for k=0,si,1 do
      rtn=lt[k]-rt[k];
      if(rtn~=0)then break;end
    end 
    return rtn;
  end
  --
  function NQueen:rbits(byte,sz)
    local score=0;
    for i=sz,0,-1 do
      if bit.band(bit.arshift(byte,i), 1) ==0 then
        score=score+2^i;
      end
    end
    return score;
  end
  --
  function NQueen:symmetryOps(si)
    local nEquiv;
    --// 回転・反転・対称チェックのためにboard配列をコピー
    for i=0,si,1 do self.trial[i]=self.board[i]; end
    --//時計回りに90度回転
    self:rotate(self.trial,self.scratch,si);    
    local k=self:intncmp(self.board,self.scratch,si);
    if(k>0)then return; end
    if(k==0)then nEquiv=2;
    else
      --//時計回りに180度回転
      self:rotate(self.scratch,self.trial,si);  
      k=self:intncmp(self.board,self.trial,si);
      if(k>0)then return; end 
      if(k==0)then nEquiv=4;
      else
        --//時計回りに270度回転
        self:rotate(self.trial,self.scratch,si);
        k=self:intncmp(self.board,self.scratch,si);
        if(k>0) then return; end 
        nEquiv=8;
      end 
    end 
    --// 回転・反転・対称チェックのためにboard配列をコピー
    for i=0,si,1 do
      self.scratch[i]=self.board[i];
    end
    --//垂直反転
    self:vMirror(self.scratch,self.trial,si);   
    k=self:intncmp(self.board,self.trial,si);
    if(k>0)then return; end 
    if(nEquiv>2)then    --//-90度回転 対角鏡と同等       
      self:rotate(self.trial,self.scratch,si);
      k=self:intncmp(self.board,self.scratch,si);
      if(k>0)then return; end 
      if(nEquiv>4)then  --//-180度回転 水平鏡像と同等
        self:rotate(self.scratch,self.trial,si);
        k=self:intncmp(self.board,self.trial,si);
        if(k>0)then return; end
        --//-270度回転 反対角鏡と同等
        self:rotate(self.trial,self.scratch,si);
        k=self:intncmp(self.board,self.scratch,si);
        if(k>0)then return; end 
      end 
    end 
    if(nEquiv==2)then self.COUNT2=self.COUNT2+1;end 
    if(nEquiv==4)then self.COUNT4=self.COUNT4+1;end 
    if(nEquiv==8)then self.COUNT8=self.COUNT8+1;end
  end
  function NQueen:backTrack2(min,left,down,right) 
    local BIT;
    local bitmap=bit.band(self.MASK,self:rbits(bit.bor(left,down,right ),self.size-1));
    if min==self.size then
      --self.TOTAL=self.TOTAL+1 ;
      if bitmap==0 then
        self.board[min]=bitmap;
        self:symmetryOps(self.size);
      end
    else
      if min < self.BOUND1 then
        bitmap=bit.bor(bitmap,self.SIDEMASK);
        bitmap=bit.bxor(bitmap,self.SIDEMASK);
      end
      if min==self.BOUND2 then
        if bit.band(down,self.SIDEMASK) ==0 then
          return;
        end
        if bit.band(down,self.SIDEMASK)~=self.SIDEMASK then
           bitmap=bit.band(bitmap,self.SIDEMASK)
        end
     end
      while bitmap~=0 do
        BIT=bit.band(-bitmap,bitmap);
        self.board[min]=BIT;
        bitmap=bit.bxor(bitmap,BIT);
        self:backTrack2(min+1,bit.lshift(bit.bor(left,BIT),1),bit.bor(down,BIT),bit.rshift(bit.bor(right,BIT),1));
      end
    end
  end
  --
  function NQueen:backTrack1(min,left,down,right) 
    local BIT;
    local bitmap=bit.band(self.MASK,self:rbits(bit.bor(left,down,right ),self.size-1));
    if min==self.size then
      --self.TOTAL=self.TOTAL+1 ;
      if bitmap==0 then
        self.board[min]=bitmap;
        --self:symmetryOps(self.size);
        self.COUNT8=self.COUNT8+1;
      end
    else
      if min<self.BOUND1 then
        bitmap=bit.bor(bitmap,2);
        bitmap=bit.bxor(bitmap,2);

      end
      while bitmap~=0 do
        BIT=bit.band(-bitmap,bitmap);
        self.board[min]=BIT;
        bitmap=bit.bxor(bitmap,BIT);
        self:backTrack1(min+1,bit.lshift(bit.bor(left,BIT),1),bit.bor(down,BIT),bit.rshift(bit.bor(right,BIT),1));
      end
    end
  end
  --
  function NQueen:NQueens(min) 
    local BIT;
    self.TOPBIT=bit.lshift(1,(self.size-1));
    self.board[0]=1;
    for BOUND1=2,self.size-1,1 do
      BIT=bit.lshift(1,BOUND1);
      self.board[1]=BIT;
      self.BOUND1=BOUND1;
      self:backTrack1(2,bit.lshift(bit.bor(2,BIT),1),bit.bor(1,BIT),bit.rshift(BIT,1));

    end
    self.LASTMASK=bit.bor(self.TOPBIT,1);
    self.SIDEMASK=self.LASTMASK;
    self.ENDBIT=bit.rshift(self.TOPBIT,1);
    self.BOUND2=self.size-2;
    for BOUND1=1,self.BOUND2,1 do
      BIT=bit.lshift(1,BOUND1);
      self.board[0]=BIT;
      self.BOUND1=BOUND1;
      self:backTrack2(1,bit.lshift(BIT,1),BIT,bit.rshift(BIT,1));
      self.LASTMASK=bit.bxor(self.LASTMASK,bit.bor(bit.rshift(self.LASTMASK,1),bit.lshift(self.LASTMASK,1)));
      self.ENDBIT=bit.rshift(self.ENDBIT,1);
      self.BOUND2=self.BOUND2-1;
    end
  end
  --
  function NQueen:NQueen()
    local max=17;
    print(" N:            Total       Unique    hh:mm:ss");
    for si=2,max,1 do
      self.size=si;
      self.TOTAL=0;
      self.UNIQUE=0;
      self.COUNT2=0;
      self.COUNT4=0;
      self.COUNT8=0;
      for k=0,self.size-1,1 do self.board[k]=k; end --テーブルの初期化
      self.MASK=bit.lshift(1,self.size)-1;    
      s=os.time();
      self:NQueens(0);
      self.TOTAL=self.COUNT2*2+self.COUNT4*4+self.COUNT8*8;
      self.UNIQUE=self.COUNT2+self.COUNT4+self.COUNT8;
      print(string.format("%2d:%17d%13d%12s",si,self.TOTAL,self.UNIQUE,self:secstotime(os.difftime(os.time(),s)))); 
    end
  end
  return setmetatable( this,{__index=NQueen} );
end
  --
NQueen.new():NQueen();


# 12.最適化 

In [ ]:
#!/usr/bin/env luajit

--[[
  Luaで学ぶアルゴリズムとデータ構造  
  ステップバイステップでＮ−クイーン問題を最適化
  一般社団法人  共同通信社  情報技術局  鈴木  維一郎(suzuki.iichiro@kyodonews.jp)
  
 １２．バックトラック＋ビットマップ＋対称解除法＋枝刈りと最適化＋対称解除法のビッ トマップ化＋クイーンの位置による振り分け（BOUND1+BOUND2)＋枝刈り＋最適化

 	実行結果

 N:            Total       Unique    hh:mm:ss
 2:                0            0    00:00:00
 3:                0            0    00:00:00
 4:                2            1    00:00:00
 5:               10            2    00:00:00
 6:                4            1    00:00:00
 7:               40            6    00:00:00
 8:               92           12    00:00:00
 9:              352           46    00:00:00
10:              724           92    00:00:00
11:             2680          341    00:00:00
12:            14200         1787    00:00:00
13:            73712         9233    00:00:00
14:           365596        45752    00:00:00
15:          2279184       285053    00:00:03
16:         14772512      1846955    00:00:20
17:         95815104     11977939    00:02:13

  ]]--

NQueen={}; NQueen.new=function()
  -- 
  local this={
    size=0;
    UNIQUE=0;
    MASK=0;
    COUNT2=0;COUNT4=0;COUNT8=0;
    BOUND1=0;
    BOUND2=0;
    TOPBIT=0;
    ENDBIT=0;
    SIDEMASK=0;
    LASTMASK=0;
    board={};trial={};scratch={};
    BIT=0;
  };
  --
  function NQueen:secstotime(secs)
    sec=math.floor(secs);
	  if(sec>59) then
		  local hour = math.floor(sec*0.000277777778)
		  local minute = math.floor(sec*0.0166666667) - hour*60
		  sec = sec - hour*3600 - minute*60
		  if(sec<10)then sec = "0"..sec end
		  if(hour<10)then hour = "0"..hour end
		  if(minute<10)then minute = "0"..minute end
		  return hour..":"..minute..":"..sec
	  end
	  if(sec<10)then sec = "0"..sec end
	  return "00:00:"..sec
  end 
  --
  function NQueen:rbits(byte,sz)
    local score=0;
    for i=sz,0,-1 do
      if bit.band(bit.arshift(byte,i), 1) ==0 then
        score=score+2^i;
      end
    end
    return score;
  end
  --
  function NQueen:symmetryOps()
    --90度回転
    local sizeE=self.size-1; --SIZEE
    local board=self.board; --array board[]
    local TOPBIT=self.TOPBIT; --TOPBIT
    local ENDBIT=self.ENDBIT; --ENDBIT
    local BOUND1=self.BOUND1; --BOUND1
    local BOUND2=self.BOUND2; --BOUND2
    --グローバルへ変更
    --local BIT=self.BIT;
		if board[BOUND2]==1 then 
			local own=1; 
      local ptn=2; 
      while own<=sizeE do
        self.BIT=1; 
        local you=sizeE; 
        while board[you]~=ptn and board[own]>=self.BIT do
          --self.B=(self.B<<1);
          self.BIT=bit.lshift(self.BIT,1);
          you=you-1;
        end
        if board[own]>self.BIT then return; end
        if board[own]<self.BIT then break; end
        own=own+1;
        --ptn=(ptn<<1);
        ptn= bit.lshift(ptn,1);
      end
		--//90度回転して同型なら180度/270度回転も同型である
      if own>sizeE then
        self.COUNT2=self.COUNT2+1;
        return;
      end
    end
    --//180度回転
    if board[sizeE]==ENDBIT then
      local own=1; 
      local you=sizeE-1;
      while own<=sizeE do
        self.BIT=1; 
        local ptn=TOPBIT;
        while ptn~=board[you] and board[own]>=self.BIT do
          --self.B=(self.B<<1);
          self.BIT=bit.lshift(self.BIT,1);
          --ptn=(ptn>>1);
          ptn=bit.rshift(ptn,1);
        end
        if board[own]>self.BIT then return; end
        if board[own]<self.BIT then break; end
        own=own+1;
        you=you-1;
      end
    --	//90度回転が同型でなくても180度回転が同型である事もある
      if own>sizeE then
        self.COUNT4=self.COUNT4+1;
        return;
      end
    end
    --	//270度回転
    if board[BOUND1]==TOPBIT then
      local own=1; 
      --local ptn=(TOPBIT>>1); 
      local ptn=bit.rshift(TOPBIT,1);
      while own<=sizeE do
        self.BIT=1; 
        local you=0;
        while board[you]~=ptn and board[own]>=self.BIT do
          --BIT=(BIT<<1);
          self.BIT=bit.lshift(self.BIT,1);
          you=you+1;
        end
        if board[own]>self.BIT then return; end
        if board[own]<self.BIT then break; end
        own=own+1;
        --ptn=(ptn>>1);
        ptn=bit.rshift(ptn,1);
      end
    end
    self.COUNT8=self.COUNT8+1;
  end   
  --
  function NQueen:backTrack2(min,left,down,right) 
    -- グローバルへ変更
    --local BIT;
    local bitmap=bit.band(self.MASK,self:rbits(bit.bor(left,down,right ),self.size-1));
    -- 枝刈りと最適化
    if min==self.size-1 then
    --if min==self.size then
      --self.TOTAL=self.TOTAL+1 ;
      --if bitmap==0 then
      if bitmap~=0 then
        -- 枝刈りと最適化
        if bit.band(bitmap,self.LASTMASK)==0 then
          self.board[min]=bitmap;
          --今回の新バージョンのsymmetryOps()
          self:symmetryOps(self.size);
          --旧バージョンのsymmetryOps()
          --self:_symmetryOps(self.size);
        end
      end
    else
      if min < self.BOUND1 then
        --BM=(BM|SM);
        bitmap=bit.bor(bitmap,self.SIDEMASK);
        --BM=(BM~SM);
        bitmap=bit.bxor(bitmap,self.SIDEMASK);
      end
      if min==self.BOUND2 then
        --if(d&SM)==0 then return; end 
        if bit.band(down,self.SIDEMASK) ==0 then
          return;
        end
        --if(d&SM)~=SM then BM=(BM&SM); end
        if bit.band(down,self.SIDEMASK)~=self.SIDEMASK then
           bitmap=bit.band(bitmap,self.SIDEMASK)
        end
     end
      while bitmap~=0 do
        -- 最適化と枝刈り インラインに最適化
        --self.aB[y],self.B=(-BM&BM),(-BM&BM);
          --BIT=bit.band(-bitmap,bitmap);
          --self.board[min]=BIT;
        self.board[min],self.BIT=bit.band(-bitmap,bitmap),bit.band(-bitmap,bitmap);
        --BM=(BM~self.aB[y]);
        bitmap=bit.bxor(bitmap,self.BIT);
        --self:backTrack2(y+1,(l|self.B)<<1,(d|self.B),((r|self.B)>>1));
        self:backTrack2(min+1,bit.lshift(bit.bor(left,self.BIT),1),bit.bor(down,self.BIT),bit.rshift(bit.bor(right,self.BIT),1));
      end
    end
  end
  --
  function NQueen:backTrack1(min,left,down,right) 
    -- グローバルへ変更
    -- local BIT;
    local bitmap=bit.band(self.MASK,self:rbits(bit.bor(left,down,right ),self.size-1));
    -- 最適化と枝刈り
    -- if min==self.size then
    if min==self.size-1 then
      --self.TOTAL=self.TOTAL+1 ;
      -- 最適化と枝刈り
      -- if bitmap==0 then
      if bitmap~=0 then
        self.board[min]=bitmap;
        --self:symmetryOps(self.size);
        self.COUNT8=self.COUNT8+1;
      end
    else
      if min<self.BOUND1 then
        bitmap=bit.bor(bitmap,2);
        bitmap=bit.bxor(bitmap,2);
      end
      while bitmap~=0 do
        -- 最適化と枝刈り インラインに最適化
        --self.aB[y],self.B=(-BM&BM),(-BM&BM)
            --BIT=bit.band(-bitmap,bitmap);
            --self.board[min]=BIT;
        self.board[min],self.BIT=bit.band(-bitmap,bitmap),bit.band(-bitmap,bitmap);
        --BM=(BM~self.aB[y]);
        bitmap=bit.bxor(bitmap,self.BIT);
        --self:backTrack1(y+1,(l|self.B)<<1,(d|self.B),(r|self.B)>>1);
        self:backTrack1(min+1,bit.lshift(bit.bor(left,self.BIT),1),bit.bor(down,self.BIT),bit.rshift(bit.bor(right,self.BIT),1));
      end
    end
  end
  --
  function NQueen:NQueens(min) 
    --グローバルへ変更
    --local BIT;
    --self.TOPBIT=(1<<(size-1));
    self.TOPBIT=bit.lshift(1,(self.size-1));
    --self.M=(1<<size)-1;    
    self.MASK=bit.lshift(1,self.size)-1;    
    self.board[0]=1;
    self.BOUND1=2;
    --forからwhileへ変更
    --for BOUND1=2,self.size-1,1 do
    while self.BOUND1>1 and self.BOUND1<self.size-1 do
        --BIT=bit.lshift(1,BOUND1);
        --self.board[1]=BIT;
      -- 最適化と枝刈り インラインに最適化
      self.board[0],self.BIT=bit.lshift(1,self.BOUND1),bit.lshift(1,self.BOUND1);
      --self:backTrack1(2,(2|self.B)<<1,(1|self.B),(self.B>>1));
      self:backTrack1(2,bit.lshift(bit.bor(2,self.BIT),1),bit.bor(1,self.BIT),bit.rshift(self.BIT,1));
      self.BOUND1=self.BOUND1+1;

    end
    -- 最適化と枝刈り インラインに最適化
    --self.SM,self.LM=(self.TB|1),(self.TB|1);
        --self.SIDEMASK=self.LASTMASK;
        --self.LASTMASK=bit.bor(self.TOPBIT,1);
		self.SIDEMASK,self.LASTMASK=bit.bor(self.TOPBIT,1),bit.bor(self.TOPBIT,1);
    --self.EB=(self.TB>>1);
    self.ENDBIT=bit.rshift(self.TOPBIT,1);
    self.BOUND1=1;
    self.BOUND2=self.size-2;
    --forからwhileへ変更
    --for BOUND1=1,self.BOUND2,1 do
    while self.BOUND1>0 and self.BOUND2<self.size-1 and self.BOUND1<self.BOUND2 do
      -- 最適化と枝刈り インラインに最適化
      --self.aB[0],self.B=(1<<self.B1),(1<<self.B1);
        --BIT=bit.lshift(1,BOUND1);
        --self.board[0]=BIT;
      self.board[0],self.BIT=bit.lshift(1,self.BOUND1),bit.lshift(1,self.BOUND1);
      --self:backTrack2(1,self.B<<1,self.B,self.B>>1);
      self:backTrack2(1,bit.lshift(self.BIT,1),self.BIT,bit.rshift(self.BIT,1));
      --self.LM=(self.LM|self.LM>>1|self.LM<<1);
      self.LASTMASK=bit.bor(self.LASTMASK,bit.rshift(self.LASTMASK,1),bit.lshift(self.LASTMASK,1));
      --self.EB=(self.EB>>1);
      self.ENDBIT=bit.rshift(self.ENDBIT,1);
      self.BOUND1=self.BOUND1+1;
      self.BOUND2=self.BOUND2-1;
    end
  end
  --
  function NQueen:NQueen()
    local max=17;
    print(" N:            Total       Unique    hh:mm:ss");
    for si=2,max,1 do
      self.size=si;
      self.TOTAL=0;
      self.UNIQUE=0;
      self.COUNT2=0;
      self.COUNT4=0;
      self.COUNT8=0;
      for k=0,self.size-1,1 do self.board[k]=k; end --テーブルの初期化
      --NQueens()へ移動
      --self.MASK=bit.lshift(1,self.size)-1;    
      s=os.time();
      self:NQueens(0);
      self.TOTAL=self.COUNT2*2+self.COUNT4*4+self.COUNT8*8;
      self.UNIQUE=self.COUNT2+self.COUNT4+self.COUNT8;
      print(string.format("%2d:%17d%13d%12s",si,self.TOTAL,self.UNIQUE,self:secstotime(os.difftime(os.time(),s)))); 
    end
  end
  return setmetatable( this,{__index=NQueen} );
end
  --
NQueen.new():NQueen();
